# Test Long Context RAG Answer Question

This notebook tests the `_answer_question` functionality in the `LongContextRAG` class.

In [ ]:
import os
import sys
from pathlib import Path
import tempfile
from loguru import logger
from autocoder.rag.long_context_rag import LongContextRAG
from autocoder.rag.rag_config import RagConfig
from autocoder.rag.cache.simple_cache import AutoCoderRAGAsyncUpdateQueue
from autocoder.rag.variable_holder import VariableHolder
from tokenizers import Tokenizer
import byzerllm
from autocoder.common import AutoCoderArgs

# Setup tokenizer
VariableHolder.TOKENIZER_PATH = "/Users/allwefantasy/Downloads/tokenizer.json"
VariableHolder.TOKENIZER_MODEL = Tokenizer.from_file(VariableHolder.TOKENIZER_PATH)
import byzerllm
llm = byzerllm.ByzerLLM.from_default_model("deepseek_chat")

In [1]:
import byzerllm
llm = byzerllm.ByzerLLM.from_default_model("sonnet_3_5_chat")
docs = "\n文档：\n\nSRE 实践白皮书\nv1.0.5\n2024年12月\nSRE-Elite.com\n![Image 1](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_1.jpg)\n![Image 1](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_1.jpg)\n修 订 记 录\n1.0.5 修订记录：\n本次新增约 3 万字\n 第三章第 5 节《故障应急》：结构进行了全面优\n化，新增 B 站和蚂蚁的两个案例\n 调整“应用服务 SLI/SLO/SLA” “重大技术保障”\n章节至本章，并在后者新增《OPPO 春节业务保障》案例；\n 大幅幅完善故障应急体系内容\n1.0.4 修订记录：\n 第三章第 2 节《研发保障》结构进行了优化， 并增\n加了《 某大型游戏全球研发保障实践》等合共 2 个案例，新\n增 2.7 万字。\n 第三章第 5 节《故障应急》结构进行了优化，依据\n2024 年 6 月 22 日 北京小米站沙龙更新并增加了《小米故障\n应急响应经验分享》等合共 5 个案例，新增 4.5 万字。\n1.0.3 修订记录：\n 第三章第 4 节《变更管理》依据 2024 年 4 月 13 日上\n海 B 站沙龙更新约 4 万字，包括 6 篇不同类型的企业案例\n1.0.2 修订记录：\n 增加了版权声明 为 CC BY-ND 4.0\n 修正了目录没有 3.1.1 的问题\n 修改了页眉的时间点\n 修正了部分错别字\n目 录\n第一章 SRE 整体介绍 ............................................ 1\n1.1 前言 ..................................................1\nSRE 发展历程 .......................................... 2\n1.2\nSRE 的目标 ............................................ 4\n1.3\n第二章 SRE 的组织架构 .......................................... 6\n第三章 SRE 的职能 ..............................................10\n1 可靠性架构设计 .........................................10\n1.1 应用韧性架构 .....................................11\n1.1.1 分布式设计 .................................11\n1.1.2 解耦设计 ...................................11\n1.1.3 冗余设计 ...................................11\n1.1.4 熔断设计 ...................................12\n1.1.5 限流设计 ...................................12\n1.1.6 降级设计 ...................................13\n1.1.7 可观测设计 .................................13\n1.2 基础设施保障 .....................................14\n1.2.1 机房多活 ...................................14\n1.2.2 网络容灾 ...................................14\n1.3 数据灾备 .........................................14\n1.3.1 数据备份 ...................................14\n1.3.2 数据回滚 ...................................14\n2 研发保障 ...............................................15\n2.1 研发保障体系设计 ..................................16\n2.1.1 代码可靠性 .................................16\n2.1.2 代码仓库可靠性 .............................28\n2.1.3 构建可靠性 .................................34\n2.1.4 制品可靠性 .................................38\n2.2 研发保障工程体系设计 .............................. 42\n2.2.1 面向研发保障的持续集成流水线 ................ 42\n2.2.2 面向研发保障的可观测设计 .................... 46\n2.2.3 面向研发保障的操作调度操作平台 .............. 48\n2.2.4 面向研发保障的 ITSM 平台 .....................51\n2.2.5 面向研发保障的容器平台 ...................... 51\n2.2.6 面向研发保障的编译加速平台 .................. 53\n2.3 研发保障案例 ......................................55\n2.3.1 腾讯游戏全球研发保障实践 ...................55\n2.3.2 某语音直播公司研发过程保障实践 .............129\n3 入网控制 ..............................................152\n3.1 运行环境适配 ....................................152\n3.1.1 运营环境设计 ..............................152\n3.1.2 容器云适配 ................................154\n3.1.3 数据库存储适配 ............................157\n3.1.4 信创适配 ..................................158\n3.2 运行环境交付 ....................................163\n3.2.1 基础资源服务 ..............................163\n3.2.2 可观测策略 ................................165\n3.2.3 自动化策略 ................................167\n3.3 测试策略 ........................................169\n3.3.1 连通性验证 ................................169\n3.3.2 功能测试 ..................................171\n3.3.3 性能压测 ..................................174\n3.3.4 数据迁移 ..................................179\n3.4 变更评审 ........................................180\n3.4.1 稳定性架构设计评估 ........................180\n3.4.2 非功能性技术评估 .........................182\n3.4.3 变更保障准备工作评估 ......................185\n3.4.4 新系统或新业务上线保障评估 ...............186\n4 变更管理 ..............................................188\n4.1 发布管理与变更管理关系阐述 .......................189\n4.2 变更体系设计 .....................................191\n4.2.1 变更体系设计原则 ..........................191\n4.2.2 变更及发布流程设计 ........................192\n4.2.3 变更的工程体系设计 ........................215\n4.3 变更管理案例 .....................................243\n4.3.1 B 站变更防控的设计与实践 ................... 243\n4.3.2 携程云平台基础设施变更管理实践 ............. 265\n4.3.3 某银行变更管理设计与实践 ................... 287\n4.4 发布管理案例 .....................................306\n4.4.1 中移互联网敏捷发布平台建设实践 ............. 306\n4.4.2 某证券变更一体化平台建设实践 ............... 325\n4.4.3 游戏 GitOps 发布管理实践 .................... 343\n5 故障应急 ..............................................350\n5.1 故障体系 .........................................350\n5.1.1 故障定义 ...................................350\n5.1.2 故障等级 ................................... 350\n5.1.3 故障序列 ................................... 351\n5.1.4 关键业务场景 ............................... 352\n5.1.5 应用服务 SLI/SLO/SLA ........................354\n5.2 故障应急体系 .....................................362\n5.2.1 故障应急体系设计原则 .......................362\n5.2.2 故障应急流程设计 ...........................362\n5.2.3 故障应急工程体系设计 .......................389\n5.3 故障应急案例 .....................................402\n5.3.1 小米故障应急响应经验分享 ...................402\n5.3.2 中国联通数字化监控平台稳定性保障实践 .......431\n5.3.3 腾讯全球化游戏故障管理实践 .................464\n5.3.4 XX 银行应急管理一体化平台建设实践 .......... 504\n5.3.5 美图故障管理体系搭建实践 ................... 518\n5.3.6 B 站轻量级容灾演练体系构建 ................. 572\n5.3.7 蚂蚁故障应急全流程体系构建及应用实践 .......611\n5.4 重大技术保障 .....................................648\n5.4.1 整体统筹保障 ..............................648\n5.4.2 技术方案保障 ..............................649\n5.4.3 工具可靠性保障 ............................650\n5.4.4 突发事件保障 ..............................652\n5.4.5 示例 1：Oppo 春节业务保障 ................. 653\n5.4.6 示例 2: 交易类大促核心保障流程和方案 ...... 668\n5.4.7 示例 3：银行类通用重大保障活动 ............ 671\n5.4.8 示例 4：发布会直播通用重大保障活动 ........ 674\n5.4.9 示例 5：哀悼日停止游戏服务保障 ............ 678\n6 上线后持续优化工作 ....................................685\n6.1 用户体验优化 ....................................685\n6.1.1 基于用户端的直接用户体验优化 ..............685\n6.1.2 基于系统端的间接用户体验优化 ..............686\n6.2 运维琐事的日常管理及优化 ........................689\n6.2.1 运维琐事的介绍 ............................689\n6.2.2 运维琐事的质量管理 ........................692\n6.3.3 运维琐事的效率管理 ........................693\n6.3 业务全生命周期工具建设 ...........................694\n6.3.1 研发期工具建设 ............................695\n6.3.2 上线期工具建设 ............................696\n6.4.3 运营期工具建设 ............................697\n6.4.4 下线期工具建设 ............................698\n6.4 运营成本分析及优化 ..............................699\n6.4.1 运营成本分析及优化的必要性 ................699\n6.4.2 运营成本实时监控 ..........................700\n6.4.3 运营成本分析及优化的指标 ..................700\n6.4.4 运营成本的统计及分析方法 ..................702\n6.4.5 运营成本的优化方法 ........................705\n6.5.5 运营成本优化持续运营 ......................708\n6.5 持续改进 ........................................710\n6.5.1 效率持续改进 ..............................710\n6.5.2 质量持续改进 ..............................712\n6.5.3 安全持续改进 ..............................713\n6.5.4 人员能力持续提升 ..........................714\n6.5.5 流程持续改进 ..............................716\n7 平台工程 ..............................................719\n7.1 标准应用平台工程建设 ............................719\n7.1.1 应用元信息平台 ............................720\n7.1.2 统一资源供给 ..............................723\n7.1.3 持续集成 ..................................724\n7.1.4 持续部署 ..................................728\n7.1.5 部署编排 ..................................731\n7.1.6 可观测 ....................................735\n7.1.7 成本（定价、用量、出账） ..................736\n7.2 异构应用平台工程建设 ............................739\n7.2.1 总体设计 ..................................740\n7.2.2\naPaaS 结构设计 ............................ 741\niPaaS 结构设计 ............................ 747\n7.2.3\n7.2.4 通用原子设计 ..............................749\n7.2.5\nSaaS 分级 ................................. 756\n7.2.6 服务管理 ..................................759\n7.2.7 安全与审计 ................................761\n附录 .........................................................766\n1 参考文献 ..........................................766\n2 术语 ..............................................766\n版权声明\n这项作品采用 CC BY-ND 4.0 许可进行授权。要查看此许可的\n副本，请访问 http://creativecommons.org/licenses/by-\nnd/4.0/\nCC BY-ND 4.0 DEED\n署名-禁止演绎 4.0 国际\n您可以自由地：\n共享 — 在任何媒介以任何形式复制、发行本作品 在任何用途下，甚至商\n业目的。\n只要你遵守许可协议条款，许可人就无法收回你的这些权利。\n惟须遵守下列条件：\n署名 — 您必须给出 适当的署名 ，提供指向本许可协议的链接，同时 标明\n是否（对原始作品）作了修改 。您可以用任何合理的方式来署名，但是不得以\n任何方式暗示许可人为您或您的使用背书。\n禁止演绎 — 如果您 再混合、转换、或者基于该作品创作 ，您不可以分发\n修改作品。\n没有附加限制 — 您不得适用法律术语或者 技术措施 从而限制其他人做许\n可协议允许的事情。\n声明：\n您不必因为公共领域的作品要素而遵守许可协议，或者您的使用被可适用\n的 例外或限制 所允许。\n不提供担保。许可协议可能不会给与您意图使用的所必须的所有许可。例\n网址：SRE-Elite.com\n微信：SRE 精英联盟\ni\n如，其他权利比如 形象权、隐私权或人格权 可能限制您如何使用作品。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\nii\n第一章 SRE 整体介绍\n1.1 前言\nGoogle 在 2003 年启动了一个全新的团队——“SRE 团队”，\n该团队旨在通过软件工程的方法提高应用系统的可靠性；随着 SRE\n相关理论和实践在 Google 的日臻成熟，SRE 实践也从 Google 慢\n慢地扩散到了整个行业。自从 SRE 的理念进入中国以来，就已经引\n起了很多企业的关注和效仿，但各企业实施 SRE 的方法各异，SRE\n的实现效果也各不相同。与此同时，中国的互联网行业中涌现出了\n一批对 SRE 充满热情的倡导者，他们为社区做出了各种贡献；包\n括：孙宇聪翻译出版了《SRE：Google 运维解密》、赵成在极客时\n间开设了课程《SRE 实战手册》，以及赵舜东在社区里积极地布道\n分享等等，不胜枚举。\n2022 年，由赵成等人牵头，首批来自于互联网、运营商、金融\n等行业领军企业的 SRE 团队负责人齐聚一堂，组织了 SRE 研讨社\n区，定期开展社区分享活动，共同探讨 SRE 在各企业里的发展路\n径，分享各自的实战经验，并总结出了这份来自一线实战的、详实\n而持续更新的《SRE 实践白皮书》。社区每年都吸纳新的成员，逐\n年更新本白皮书内容，力求真实客观地描述国内企业 SRE 团队的工\n作方式。在《实践白皮书》初稿长达两年的整理过程中，我们看到\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n1\n了不同企业对 SRE 的理解，并尽可能统一大家对相似场景的定义；\n我们看到了不同企业对 SRE 职能领地的扩展，并将成功团队的经验\n提炼成案例供大家参考；我们也看到了在这两年的编写过程中，不\n同企业 SRE 团队的真实变化，并及时将其更新到实践白皮书中。总\n之，在未来的每个季度，我们都会将各 SRE 团队的最新职能、组织\n形式、技术迭代等现状，补充到《实践白皮书》中。\n2023 年，中国信息通信研究院（下简称信通院）云计算与大数\n据研究所（下简称云大所）稳定性保障实验室的专家加入了 SRE 研\n讨社区，深度的参与到社区交流当中，为《SRE 实践白皮书》的编\n写工作提供了专业指导。\n1.2 SRE 发展历程\nSRE 运动在全球的发展经历了 20 年，下面是部分重要事件：\n2003 年，Google 成立了第一个 SRE 团队；\n2010 年，Facebook 拥有了一个 SRE 团队；\n2014 年，USENIX 协会主办的首届 SREcon（网站可靠性工程会\n议）在美国举行，大会成为了 SRE 专业人士交流经验和最佳实践的\n重要平台，标志着 SRE 作为一个独立且重要的专业领域在全球范围\n内的正式认可。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n2\n2016 年，前 Google SRE 孙宇聪翻译出版了首部中文专业书籍\n《SRE：Google 运维揭秘》，在国内引起了很大的反响，很多企业开\n始学习并成立自己的 SRE 团队；\n2016 年，Netflix 成立了“核心 SRE 团队”。Uber 开始撰写\n有关其如何使用 SRE 的文章；\n2016 年，蚂蚁集团在国内成立了第一支 SRE 团队，主要攻坚容\n灾架构，后续拓展到高可用、资金安全等多个业务风险领域；\n2017 年，LinkedIn 开始宣传其“ SRE 文化”；\n2017 年，浙江移动正式组建应用 SRE 团队，开始收口 IT 系统\n的集成部署、应急保障、架构治理等工作职责，加速了传统企业的\n运维数字化的转型进程；\n2018 年， 赵成在某次 SRE 的聚会上，拉起了 “聊聊 SRE”微\n信群， 国内 SRE 人才开始聚拢，SRE 社区初步成型，并逐步成为了\n最具影响力 SRE 中文社区；\n2021，阿里 CTO 线第一支横向 SRE 团队成立，隶属于技术风险\n与效能部，负责集团全局稳定性保障、资源成本等方面工作；\n2022 年，腾讯在内部技术岗位设置中，新增了 SRE，标志着腾\n讯内部 SRE 体系的正式成立；\n2023 年，信通院云大所稳定性保障实验室牵头编制《服务韧性\n工程（SRE）成熟度模型》标准，推动该领域深入研究与实践应用，\n并在稳定性保障实验室成立了专门的“SRE 工作组”。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n3\n1.3 SRE 的目标\nSite Reliability Engineering（SRE）的主要目标是通过结合\n软件工程和系统运维的最佳实践，提高大规模分布式系统的可靠\n性、可用性、性能和效率。以下是部分 SRE 追求的核心目标：\n可靠性： SRE 的首要目标是确保服务和系统的可靠性。这包括\n减少故障、提高系统的稳定性，以确保用户在任何时候都能够获得\n一致的高质量服务。\n可扩展性： SRE 致力于设计和实施能够随着用户需求增长而扩\n展的系统。这涉及到对系统的架构和资源进行优化，以便在不降低\n性能的情况下，适应实际工作负载持续不断的峰谷状态变化。\n性能： SRE 关注系统的性能，旨在确保系统能够在合理地时间\n内快速响应用户请求。这包括对系统瓶颈的持续监控和优化，以提\n高整体性能。\n自动化： SRE 倡导自动化运维工作，以减少人为错误和提高效\n率。通过自动化，可以更快速地部署新功能、检测并响应故障，并\n合理的开展系统的升级和维护工作。\n监控和告警： SRE 强调对系统的全面监控，以便及时发现并解\n决问题。通过设置有效的告警系统，可以在重大问题发生前迅速做\n出反应，从而减少对用户的影响。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n4\n故障恢复： SRE 强调迅速而有效地恢复服务，以最小化用户体\n验的中断。这包括制定和演练紧急情况的应急计划。\n企业实现 SRE 核心目标的过程并不相同，落地路径各异。不论\nSRE 部门（团队）在企业中的存在形式和所处位置，SRE 相关实践\n工作存在于大量流程中。这些工作流程与研发、测试、运维、产品\n运营等团队紧密的融合在一起，所有参与团队都在上述共享的 SRE\n目标上做着各自的贡献。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n5\n第二章 SRE 的组织架构\nSRE 团队在组织中的存在意义主要是确保系统的可靠性和高效\n运行。通过引入 SRE 角色，组织可以更好地平衡软件开发速率和系\n统稳定性之间的需求，从而实现更高水平的可用性、性能和自动\n化。通常 SRE 团队在组织中使命如下：\n可靠性优先： SRE 团队致力于确保服务的高可用性和可靠性。\n他们关注系统的稳定性，采取工程化方法来减少故障和提高系统的\n稳定性。\n自动化运维： SRE 团队推动自动化运维工作，以减少手动操作\n的错误和提高效率。通过自动化，可以更快速、可靠地进行部署、\n监控、故障检测和修复等操作。\n质量保证：SRE 团队参与服务的全生命周期，包括设计、开\n发、部署和维护阶段，以确保系统在不同阶段都能保持高质量。\n快速创新： 通过减少故障和提高系统的稳定性，SRE 团队为开\n发团队提供了更稳定的平台，使其能够更专注于业务创新和新功能\n的开发。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n6\n在组织架构中，SRE 团队的存在形式可以各不相同，这主要取\n决于组织的规模、业务需求和文化。以下是一些常见的 SRE 团队的\n存在形式：\n中心化 SRE 团队： 由一个专门的 SRE 团队负责支持整个组织的\n可靠性工作。这种模式有助于集中专业知识，确保在整个组织中实\n施一致的最佳实践。\n嵌入式 SRE 团队： SRE 团队成员被嵌入到各个产品或服务团队\n中，与开发团队紧密合作。这种模式有助于更好地集成可靠性工作\n到产品开发的全过程中。\n混合模式： 一些组织采取混合模式，既有中心化的 SRE 团队，\n又在一些关键项目中嵌入 SRE 角色。这种方式能够兼顾专业化和贴\n近业务的优势。\n每种存在形式都有其优势和适用场景，关键在于根据组织的需\n求选择最合适的模式。不论哪种方式，SRE 的目标都是通过自动化\n和工程方法提高系统的可靠性和效率。\n下面是国内某几家一线互联网 SRE 团队在组织架构中的设置模\n式。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n7\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n8\n![Image 3](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_3.jpg)\n![Image 3](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_3.jpg)\n参考以上各个公司 SRE 团队在组织架构中的位置，通常 SRE\n团队需要承担以下几类职责：监控、事故响应、事后回顾、测试与\n发布、容量规划、工具开发和可用性改进等。\n由于各个公司的业务形态的不同，SRE 团队在组织架构中也有\n不同的定位和名称，包括：SRE 产品运维、互联网 SRE 组、AIoT\nSRE 组、信息技术 SRE 组、业务 SRE 组等。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n9\n![Image 5](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_5.jpg)\n![Image 5](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_5.jpg)\n第三章 SRE 的职能\n1 可靠性架构设计\n可靠性架构设计是指在进行系统架构设计的过程中，根据系统\n的可靠性需求，采用分布式设计、解耦设计、冗余设计等高可靠性\n的架构设计方案，以提升系统的可靠性。\n在进行可靠性架构设计的过程中，SRE 团队需要将应用架构设\n计流程完全融入其中，并与研发团队共同参与架构设计和评审工\n作。在系统设计阶段，应尽量消除可能出现的单点、容量等潜在风\n险，并提前为可能出现的系统架构风险做好应急准备。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n10\n![Image 7](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_7.jpg)\n1.1 应用韧性架构\n1.1.1 分布式设计\n在系统中，存在被划分为职责明确、粒度合适且易于管理的组\n件，这些组件（如计算资源、业务部分、数据等）都可以进行分布\n式的部署和运行。组件之间相互独立、互不干扰，通过分布式设计\n可以提高开发效率和可靠性。组件的拆分和分布可以通过复制、根\n据功能进行垂直拆分、或根据用户与访问模式水平拆分等形式。\n在设计时应该充分考虑到组件间可能存在的相互干扰以及如何\n平衡不同组件之间的负载，并将系统所承受的压力进行均匀分配，\n以减轻压力对系统整体性能的不良影响。\n1.1.2 解耦设计\n在架构设计中，可以将各种逻辑功能划分为不同的服务模块，\n确保不同模块的故障对其他模块的影响是最小，从而最大限度地降\n低模块之间的耦合度。通过这种方式，可以将系统划分为多个相互\n独立的功能模块来实现。尤其值得注意的是，业务的主要逻辑与其\n他非核心模块是独立的，因此业务非核心模块的故障并不会对业务\n的核心功能产生负面影响。\n1.1.3 冗余设计\n为了确保资源有足够的安全余量，每个组件都需要有足够和合\n理的冗余实例，以确保单一组件实例的失效不会对业务的正常运行\n造成影响。对于不同类型的组件，我们需要明确地定义冗余量和冗\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n11\n余类型。在实际应用中，由于设备故障或者操作不当等原因导致服\n务器出现性能下降或崩溃现象时，系统会出现异常状态并产生大量\n信息。应用程序可能部署多个机房，当这些机房中有数据冗余时，\n一个位置的错误可以通过另一个位置的数据进行修正，确保整个系\n统的连续性和可靠性。为了提高系统可靠性，通常采用读-写分离的\n技术进行数据的冗余管理。读写分离是一种冗余的设计方式，缓存\n和数据库之间存在数据冗余，当缓存服务宕机时，可以从数据库回\n源到缓存。\n1.1.4 熔断设计\n熔断机制是应对雪崩效应的一种微服务链路保护机制，如果目\n标服务的调用速度较慢或超时次数较多，则此时会熔断该服务的调\n用。对于后续的调用请求，不再继续对目标服务进行调用，直接返\n回预期设置好的结果，可以快速释放资源。一般来说，熔断需要设\n置不同的恢复策略，如果目标服务条件改善，则恢复。\n1.1.5 限流设计\n限流是一种系统设计技术，用于控制访问应用程序或服务的流\n量，防止资源过载。常见的限流策略包括固定窗口、滑动日志、漏\n桶和令牌桶算法。这些方法可以帮助系统应对高流量，保持稳定性\n和可靠性。在实施时，通常需要结合其他系统保护措施，如队列、\n缓存、服务降级和熔断，以实现全面的流量控制和系统保护。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n12\n当流量被限制后，系统通常会采取以下措施之一：拒绝多余的\n请求、将请求排队等待处理、返回错误码（如 HTTP 429 Too Many\nRequests）、或者提供一个降级的服务响应。这些措施可以缓解服\n务器压力。\n1.1.6 降级设计\n降级机制是当服务器压力剧增的情况下，根据当前业务情况及\n流量对一些服务和页面有策略地降级，以此缓解服务器资源的压\n力，释放服务器资源以保证核心任务的正常运行。从降级配置方式\n上，降级一般可以分为主动降级和自动降级。主动降级是提前配\n置，自动降级则是系统发生故障时，如超时或者频繁失败，自动降\n级。其中，自动降级可分为超时降级、失败次数降级、故障降级。\n1.1.7 可观测设计\n为了保证系统的透明性并迅速定位问题，采用可观测的设计方\n法变得尤为关键。可观测设计涵盖了日志记录、实时监控、追踪以\n及度量等多个方面，从而实现了系统状态和行为的可量化以及可分\n析性。在可观测的设计中，日志应当详细地记录所有的关键事件，\n监控系统需要能够实时捕获关键的性能指标，跟踪机制应具备跨服\n务请求的追踪能力，度量指标则应全方位地反映系统的健康状态。\n另外，健康检查机制需要自动地对系统组件状态进行评估，当出现\n异常指标时，告警机制会立即告知相关的工作人员。通过这些措\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n13\n施，我们可以清晰地观察到系统的运行状态，从而为后续的维护和\n优化工作奠定了稳固的基础。\n1.2 基础设施保障\n1.2.1 机房多活\n系统所部署的机器或所在地需具备一定的冗余性。包括同机房\n多活、同城多活和异地多活等不同级别。将要建设的机房要求具有\n独立性，尤其是网络环境，机房之间通过专线来进行连接。\n1.2.2 网络容灾\n数据中心之间的互联网络是 DC 之间业务连接的重要载体，对存\n储灾备网络的时延要求较低、带宽较大、可靠性较高；业务灾备网\n络需要实现链路备份和快速的路由收敛。\n1.3 数据灾备\n1.3.1 数据备份\n即对核心数据进行备份和恢复的能力。需对核心数据进行实时\n备份，并具备快速容灾切换的能力。需对备份恢复的能力进行周期\n性地验证。\n1.3.2 数据回滚\n在系统出现异常情况下，迅速有效地恢复故障前数据状态，减\n少了故障给业务系统带来的冲击。回滚是否有效取决于回滚执行过\n程和回滚决策是否及时。\n14\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n2 研发保障\n通常说法是，SRE 处置的线上可靠性问题中，有 70%左右源自\nCD 阶段（详见第三章第 4 节“变更管理”），15%左右源自研发阶\n段（例如代码的低质量、高维护成本等）。对于初创的 SRE 团队，\n应该将工作重心放在 CD 与 CO 阶段，做好救火工作，同时夯实 CD-\nCO 阶段的工程化，保障 SLA。在有阶段性成果之后，应该逐步向研\n发服务扩展进入 CI 领域，一方面尝试缩减那 15%的可靠性根因，\n更重要的是打通 CI-CD，以乙方服务心态保障业务团队的研发连续\n性，追求平台工程的研运一体化理念。\n有些企业是由质量团队或者独立的效能团队承担这一部分职\n能，也有业务研发兼职的情况，SRE 团队应该本着以往运维团队的\n服务传统，将业务研发环境视为一个独立的线上业务，以服务心态\n尝试“接管”研发环境中代码仓库、CI 流水线、各类测试环境、代\n码分析扫描平台、制品仓库等研发基础设施的运维工作，并保障其\n可靠性，进而提高业务团队的生产效率，降低其出错概率。\n有能力的 SRE 团队接下来可以依托自身的工程化能力将这些工\n具升级改进，与 SRE 前期已经建设的 CD-CO 平台整合，形成覆盖代\n码全生命周期的“研发运维运营”一体化的平台工程基础设施。这\n种发展路径借助运维 SRE 与业务开发团队的紧密关系减少了内耗，\nCI-CD-CO 全路径由 SRE 团队建设避免了轮子，还减少了例如 PE、\nDevOps 等小众的公司级岗位设置。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n15\n研发过程可靠性，指的是以 SRE 理论驱动研发连续性建设，提\n升研发管线的工业化水平，保障版本能够高质量迭代，从而持续保\n障业务的线上可靠性，同时兼顾了版本迭代效率；\nSRE 主导的研发连续性保障，可以拆解为代码可靠性、代码仓\n库可靠性、构建可靠性和制品可靠性四个方面，每个阶段分别对其\n可靠性进行定义并提出相应的改善措施；\n2.1 研发保障体系设计\n2.1.1 代码可靠性\n代码是基于一定需求实现，用于构建对应软件的文件集合。代\n码质量从基础上决定了软件的成败，是软件开发过程中不可忽视的\n一环。在软件版本快速迭代的今天，如何构建高质量的代码更显得\n至关重要。\n代码可靠性的落地仅靠宣导或者文档还远远不够，需要建设完\n善的检查工具并量化效果，一般由平台 SRE 建设相关的工具，因此\n需要平台 SRE 需要深入了解影响代码可靠性的常见问题及提升措\n施，不断完善代码检查工具的能力；\n2.1.1.1 代码缺陷\n代码缺陷是指影响代码稳定运行的问题，或者未达到设计时的\n预期功能。缺陷产生的原因有多种，比如：软件的复杂性、编写的\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n16\n错误、需求歧义等。代码缺陷的及时发现与修复，对项目进度与工\n程质量至关重要。\n代码缺陷检测，一般可采用静态分析法或动态分析法。\n静态代码分析是指无需运行被测代码，通过词法分析、语法分\n析、控制流、数据流分析等技术对程序代码进行扫描，找出代码隐\n藏的错误和缺陷，如参数不匹配，有歧义的嵌套语句，错误的递\n归，非法计算，可能出现的空指针引用等等。\n动态分析方法则一般应用于软件的测试运行阶段，在软件程序\n运行过程中，通过分析动态调试器中程序的状态、执行路径等信息\n来发现缺陷。\n1．代码缺陷规避措施\n代码缺陷种类较多，无法完全罗列，这里选取部分最为常见的\n缺陷并介绍对应的规避办法：\n1）指针错误使用\n指针的错误使用，一般要避免空指针、野指针，避免指针类型\n不匹配，不返回局部变量的指针。\n2）内存非法访问\n非法内存访问是指程序试图读取或写入未分配/受保护的内存，\n如数组越界等，这将会导致程序的不可控行为。因此，必须确保程\n序正确地分配和释放内存，避免缓冲区溢出等现象。也可使用静态\n分析工具检测代码中的内存非法访问问题。\n3）变量未初始化\n17\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n使用未初始化的变量，可能导致未知错误。一般来讲，变量需\n要在声明时赋予初始值。\n4）资源泄漏\n常见的资源泄漏包含 socket 泄漏，文件句柄泄漏，内存泄漏\n等。产生原因是由于未能正确释放已经分配的内存或其他资源，导\n致这些资源被长期占用。资源泄漏不仅会造成资源的浪费，系统性\n能下降，严重时超出系统限制会导致程序崩溃。\n避免资源泄漏，在编程过程中，需要在资源使用完毕后进行资\n源的释放，如 socket/文件句柄的关闭，动态分配内存的释放。\n5）竞争死锁\n竞争死锁是指多个线程或进程持有资源，又互相竞争等待对方\n资源而导致死锁的情况。解决死锁问题，一般可采用超时使用机\n制、统一获取资源顺序和死锁检测机制来打破死锁产生的必要条\n件。\n6）不当的 API 使用\n不当的 API 使用，会导致程序异常。可通过仔细阅读 API 文\n档，了解 API 的使用方式，在使用 API 前进行充分测试的方法来规\n避。\n2．效果评估\n代码缺陷数作为代码质量指标之一，可以从数量、严重程度来\n归类。在度量层面，一般使用百行告警数来衡量代码缺陷指标。通\n过配置代码缺陷规则集，采用缺陷检测工具扫描，生成检测报告。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n18\n根据缺陷的严重程度分为严重告警（空指针、数组越界等），\n一般告警（变量未初始化等）和提示告警（如代码风格等）。设定\n各个告警等级的权重，统计代码行数，最终计算出百行代码缺陷告\n警数。\n百行缺陷告警数=（严重告警 W1+一般告警 W2+\n提示告警 W3）/代码行数 100\n（W1/W2/W3 为权重系数）\n2.1.1.2 代码规范\n代码规范主要是指是否遵守了团队或者业界的编码规范。代码\n规范主要涵盖：代码风格（如注释、空代码块、命名、格式化\n等），与异常处理等部分。\n代码规范有助于提高代码可读性与可维护性，从而提升团队内\n开发效率。代码可读性帮助相关技术人员能够轻松阅读并理解代码\n意图与实现方式。代码从分支发起到主干的合并请求前，必须进行\n代码检查，这也是提前发现问题的方法之一。\n1．代码规范提升措施\n1）代码风格\n良好的代码风格会帮助开发人员阅读和理解符合该风格的源代\n码，并且避免错误。此处所讲述的代码风格包括但不限于：命名规\n范，表达式与语句，缩进，对齐，注释，代码布局等。对于不同的\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n19\n编程语言，适用于不同的代码风格，对于同一项目或开发团队，应\n当使用统一的代码风格。\n（1）命名规范，命名要能够直观的表达本身的意图，同时具备\n可读与可搜索性，尽量遵循一些通用的写法。在此基础上命名长度\n应当尽量精短，避免触发代码行字符数限制规范；\n（2）缩进，一般采用 4 个空格缩进，而不使用 tab 键（特殊语\n言除外）\n（3）统一字符编码格式，通常采用 UTF-8 编码\n（4）单行字符数限制，长度一般不超过 120\n（5）行尾换行符，一般使用换行符 LF，禁止使用回车键 CR\n2）异常处理\n异常处理是为了防止一些未知错误产生而采取的措施。适当的\n使用异常处理能够提高程序的容错性。在处理异常方面，需要遵\n循：\n（1）只在可能出异常的块进行精准捕获处理；\n（2）捕获的异常必须处理或抛出给上层调用方；\n（3）异常处理效率较低，应避免使用异常做条件控制\n2．效果评估\n对于代码规范的效果评估，可以采用百行告警数来衡量。\n百行告警数=（严重告警 W1+一般告警 W2+\n提示告警 W3）/代码行数 100\n注：（W1/W2/W3 为权重系数）\n20\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n百行告警数可以用来评估代码的质量和稳定性，较高的百行告\n警数可能意味着代码存在较多的缺陷和潜在问题，需要更多的测试\n和修复工作；\n2.1.1.3 代码安全\n安全性是指在为正常访问提供服务的同时，也能拒绝非法访\n问。同时不因为代码设计或实现的原因，导致信息泄漏/非法侵入/\n系统崩溃等问题。\n1．代码安全性提升措施\n1）防止敏感信息泄漏\n敏感信息可分为系统敏感信息与应用敏感信息。系统敏感信息\n包含业务系统的基础环境信息，如系统版本、组件版本等；应用敏\n感信息包含用户信息和应用信息等，如用户 TOKEN、密码、IP 等。\n系统敏感信息泄漏会为攻击者提供更多的攻击方法，应用敏感信息\n泄漏危害则因泄漏信息内容而决定。\n解决方法：\n（1）避免硬编码，禁止将密码等敏感信息写入到代码，应该以\n配置或后台下发形式读取\n（2）处理异常时，避免将系统信息、DEBUG 信息、或者敏感文\n件的路径输出到用户可见处\n2）预防安全漏洞\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n21\n代码安全漏洞，指编码过程中因不当的处理逻辑引发的安全风\n险；\n常见的代码安全漏洞有：\n（1）脚本（SQL）注入，可通过减少拼接命令，对命令参数值进\n行过滤/校验避免\n（2）XSS 攻击，可以使用安全的 JavaScript 框架和组件，同时\n主动检测发现，转义输入等减少\n（3）越权访问，是由于权限设计错误，未授权用户获取甚至修\n改其他用户的信息。需要通过最小化原则的权限设计与审计来规避\n（4）通信安全，一般是由于未使用加密信道进行通信导致，可\n以通过使用加密/私有协议通信来避免\n3）第三方组件安全\n软件开发中不可避免的会引入依赖库，或者第三方 SDK。这些\n第三方组件作为系统的一部分，与原生代码并无本质区别，它们的\n安全性也同样影响整个系统。因此，需要在减少对第三方组件引入\n的基础上，加入相应的安全评估机制。\n对于第三方组件的评估，我们主要从以下几个方面：\n（1）组件安全风险应在引入前/上线前/定期进行安全扫描\n（2）组件合规风险包括使用协议合规和监管数据合规\n（3）组件稳定性应当使用经过实际验证的 LTS 版本\n2．效果评估\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n22\n在衡量代码安全性方面，可以从敏感信息泄漏、系统漏洞、第\n三方高危组件等几个方面来考量。可以通过代码扫描工具，扫描出\n已知系统漏洞与敏感信息，以及第三方高危组件的引入情况。\n在得到敏感信息，安全漏洞个数，以及第三方高危组件个数\n后，可以制定代码安全性红线。一般来讲，敏感信息、安全漏洞是\n绝对不允许的。对于第三方高危组件，也要经过安全评估测试，其\n标准与本身代码相同。\n（1）对于敏感信息与安全漏洞，必须彻底清除\n（2）第三方高危组件，还可以采用 LTS 覆盖率可以用来衡量系\n统中使用的第三方高危组件稳定性。LTS 覆盖率=第三方高危组件\nLTS 数/第三方高危组件数\n2.1.1.4 代码圈复杂度\n圈复杂度是一种代码复杂度的衡量标准，可以用来衡量一个模\n块流程判定结构的复杂程度。圈复杂度大说明程序代码的判断逻辑\n复杂，可用来表示对给定代码进行测试、维护或故障排除的难度，\n以及代码生成错误的可能性。同时，也可用来帮助开发人员确定是\n否需要对程序进行重构，以降低程序的复杂度，提高代码质量。\n1．圈复杂度改善措施\n降低函数圈复杂度的主要通过对代码重构来进行，一般有以下\n几种方法：\n（1）将大函数拆分成多个小函数，每个小函数只负责单一功能\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n23\n（2）将条件判定提炼出来，成为独立函数\n（3）简化、合并条件表达式\n（4）优化循环结构，减少循环嵌套、使用更简单的循环结构等\n2．效果评估\n圈复杂度反应了代码的耦合度，圈复杂度越高的代码会有越多\n潜在的 BUG。\n对于圈复杂度，可以从以下指标衡量：\n（1）单函数圈复杂度最大值小于等于 20\n（2）项目平均圈复杂度，一般不大于 4\n项目平均圈复杂度=所有函数圈复杂度之和/所有函数个数\n2.1.1.5 代码重复\n代码重复指的是程序中存在相同或类似的代码段。在不同的位\n置或程序中出现相同的代码，会造成了代码冗余和浪费。代码重\n复，不仅导致项目代码量的增加，影响程序的可读性和可维护性，\n增加代码的错误率和修改难度，也是设计不佳的一个标志。\n代码重复的表现形式多种多样，常见形式有：\n（1）完全一样的代码\n（2）仅重命名标识符的代码\n（3）仅变量赋值不一样的代码\n（4）插入或删除语句的代码\n（5）重新排列语句的代码\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n24\n1．代码重复改善措施\n降低重复代码，是代码优化的重要方面之一，一般需要对相关\n功能进行重构。\n常见的改善措施，主要是抽取公共代码、封装函数、使用继承\n和多态等\n（1）抽象出公共方法或函数，将重复的代码封装在一个函数或\n方法中\n（2）使用继承或接口，将共同代码放在父类或接口中，子类只\n实现自己的特定部分\n（3）使用设计模式，如工厂模式、模板方法模式等\n（4）利用现有的框架或库，避免自己重写\n2．效果评估\n代码重复的度量，可以使用代码重复率来表示，可以通过静态\n扫描工具得出。\n代码重复率，指的是在一段代码中重复出现的代码段的比例。\n代码重复率越高，代码的可维护性和可读性就越差。\n代码重复率=重复行数/代码总行数\n在实际的工程中，一般建议：\n（1）单文件代码重复率最大值小于等于 10%\n（2）项目平均代码重复率小于等于 10%\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n25\n2.1.1.6 代码注释与 API 文档\n代码的注释与 API 文档编写是软件开发过程中非常重要的一部\n分，可以提高代码的可读性和可维护性。通过代码注释可以帮助阅\n读者快速理解代码的功能和实现方式。API 文档则是其他开发者了\n解使用软件的重要途径。开发者应该养成写注释和文档的好习惯，\n为自己和其他开发者节省开发时间。\n1．代码注释与 API 文档提升措施\n（1）注释目的在于使阅读者能够快速掌握注释对象的使用方式\n与原理，良好的注释应包\n含注释对象的产生意图，设计考量与如何使用；注释一般应当\n包含文件注释，类/结构体注释，函数方法注释，变量注释以及适当\n的代码段注释；对于规范化命名的变量与简单函数方法，可以不进\n行注释。\n（2）API 文档，应当描述各个类和方法的功能和使用方法，同\n时遵循行业和国际标准，具备兼容性和实时性。\n2．效果评估\n对于注释与 API 文档的考量，可以从 API 文档覆盖率，代码注\n释行密度来衡量。\n1）注释行密度=注释行数/总行数* 100\n此标准用于衡量百行代码中，所包含注释行数，一般认为低于\n5 表示几乎没注释。\n2）API 文档覆盖率=已覆盖 API 接口数量/总 API 接口数量\n26\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n此标准用于衡量对外 API 文档的完善程度；一般来讲，至少需\n要达到 80%覆盖率，即覆盖大部分的 API 功能，忽略了一些不太重\n要或不常用的 API；同时也需要定期更新，使文档保持最新、全\n面、准确的状态。\n2.1.1.7 代码质量红线\n代码质量红线是指在开发过程中，开发团队所设定的一些规则\n和标准，用于确保代码质量达到一定的水平，也是衡量代码质量的\n一个综合考量。这些规则和标准通常是基于行业最佳实践和经验总\n结制定的，是团队开发的一种约束和保障。当代码质量超越红线\n时，就需要开发团队及时进行修正和优化，以确保代码质量和运行\n稳定性。\n1．质量红线改善措施\n质量红线的触发标准是基于每个质量指标的。提升每项指标质\n量有助于避免触发红线，也可以帮助开发团队提高软件开发的效率\n和质量，减少错误。\n2．效果评估\n质量红线一般包含以下几个指标：\n（1）代码缺陷\n（2）代码风格\n（3）代码安全性\n（4）圈复杂度\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n27\n（5）代码重复率\n（6）代码注释和文档\n（7）单元测试覆盖率\n通过在代码合并、转测等场景下，对以上每个指标单独设定阈\n值，可以划定出不同场景下的质量红线，当某项指标触发质量红线\n时终止后续的 CI/CD 流程，并要求开发团队进行修复。\n2.1.2 代码仓库可靠性\n代码仓库就是存放源代码和资源的地方，亦称版本库、代码\n库，其核心功能是版本控制，记录一个或若干文件的变化，以便后\n续查看特定版本修订的情况;\n代码仓库出现问题，对代码拉取、项目开发、编译构建等都会\n造成影响，所以代码仓库的可靠性是整体研发流程可用性的关键一\n环；\n代码仓库的可靠性包括：仓库性能、仓库容灾、仓库安全和仓\n库可扩展性四个方面；\n代码仓库的可靠性主要侧重网络优化、部署优化、配置优化、\n安全提升等等工作，可由服务 SRE 和安全人员来承担相关能力的建\n设；\n2.1.2.1 仓库性能\n代码仓库的性能通常是指代码仓库对代码的存储、管理和处理\n�1�的速度和效率，包括代码提交和拉取的速度、分支合并的速度\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n28\n等，高性能的代码仓库，可以减少开发人员的等待时间，缩短产品\n交付周期；\n1．代码仓库性能提升措施\n（1）控制代码仓库的大小：代码库的大小会直接影响仓库的性\n能，因为大型代码仓库需要更多的时间来处理和查找文件。因此，\n需要合理控制仓库大小，及时删除不需要使用的文件，并合理设置\n文件的保存周期；\n（2）合理设置代码仓库结构：如果代码仓库的结构合理，可以\n更快地查找和访问文件，从而提高性能，例如，某些版本管理软\n件，支持大文件单独存储在仓库之外，仓库中实际只存储一个很小\n的文本指针，可以将存储大文件的目录设置使用更适合的存储形\n式；\n（3）版本工具选型：不同的版本控制工具可能会对性能产生不\n同的影响。例如，Git 和 SVN、P4 的架构和设计理念的差异，在处\n理大文件的性能存在差异，需要根据业务资源文件的数量和大小、\n团队的多地分布特性等综合选择；\n（4）网络优化：如果多个人同时访问代码仓库，网络连接的速\n度也会影响性能，在评估代码仓库性能时，需要考虑网络连接的速\n度和质量，可在离用户就近的网络区域，部署边缘节点，缓存最近\n的版本，减少网络距离传输损耗，提高访问速率；\n（5）硬件升级：硬件也会影响代码仓库的性能，例如，使用较\n高 IO 性能的磁盘，可以提高代码仓库的读取速度；\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n29\n（6）集群化：通过集群化来部署代码仓库，可以让代码仓库支\n持更大规模团队的使用；\n2．效果评估\n一般采用下面 3 个指标来评估仓库的性能\n（1）文件下载速度：通常是指每秒传输的数据量，常见的单位\n有比特/秒（bps）、千比特/秒（Kbps）、兆比特/秒（Mbps）和千\n兆比特/秒（Gbps）等；\n（2）下载卡顿率：是指从仓库中下载时出现卡顿的频率或时间\n占比，通常使用百分比（%）来表示；\n（3）并发请求量：一般团队多人同时拉取或者提交代码可能会\n影响速度，通常使用 QPS 来表示系统每秒钟的请求量；\n2.1.2.1 仓库容灾\n代码仓库容灾是指代码仓库在经受自然灾害、设备故障、网络\n故障、人为错误等不可预测的问题后，通过备份、容错机制和恢复\n策略在最短时间内恢复到正常可用的状态。完备的代码仓库容灾机\n制，可以避免团队或公司核心代码资产遭受损失。\n1．代码仓库容灾提升措施\n（1）数据备份：定期对代码仓库的数据进行备份，确保在数据\n丢失或损坏时能够及时恢复。\n（2）多地备份：将备份数据存储在多个地方，以防止单点故\n障。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n30\n（3）容错机制：使用容错技术，如 RAID 等，以防止硬件故障导\n致数据丢失。或者将本地普通硬盘替换为云硬盘，云硬盘中的数据\n以多副本冗余方式存储，会避免数据的单点故障风险。\n（4）灾备恢复策略：制定灾备恢复策略，以便在发生灾难时能\n够及时恢复。\n（5）人员培训：对相关人员进行培训，提高应对灾难的能力和\n应变能力。\n2．效果评估\n一般使用以下几个指标来评估代码仓库容灾效果\n（1）恢复时间目标（RTO）：Recovery Time Objective，他是\n指故障发生时间到故障恢复时间，两个时间点之间的时间段称为\nRTO；\n（2）恢复点目标（RPO）：Recovery Point Objective，是指系\n统恢复到怎样的程度。这种程度可以是上一周的备份数据，也可以\n是上一次的实时数据；\n（3）投入产出比（ROI）：Return of Investment，容灾系统的\n投入产出比，可以使用最高的性价比方案来达到容灾效果，为团队\n节省成本；\n2.1.2.3 仓库安全\n代码仓库的安全性是指代码仓库中存储的代码等数据受到保护\n的程度，以防止未经授权的访问、篡改、泄露和破坏。保护代码仓\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n31\n库的安全性包括但不限于访问控制、数据加密、代码审查、安全漏\n洞、操作审计、私有网络部署等。高安全性的代码仓库可以保护代\n码的机密性，完整性，避免因安全漏洞造成团队或者公司的损失和\n风险。\n1．代码仓库安全性提升措施\n（1）访问控制：评估代码仓库中代码的访问控制机制，包括用\n户认证、授权、权限管理等，确保只有授权的用户能够访问仓库中\n的代码。\n（2）数据加密：评估仓库中存储的关键元数据或者敏感代码是\n否采用了合适的加密技术进行保护，以防止敏感信息泄露。\n（3）代码审查：评估代码审查机制，确保代码质量和安全性。\n详细可参考：4.2.1.3 代码安全。\n（4）安全漏洞：评估代码仓库中可能存在的安全漏洞，包括代\n码中的漏洞、第三方库中的漏洞等。\n（5）操作审计：评估代码仓库的操作审计，确保所有操作可追\n踪，以便及时发现和应对安全事件。\n（6）私有化部署：将代码仓库部署在私有网络中，例如，企\n业、学校的内部网络中，相比公网中的外部代码托管服务理论上会\n更安全；\n2．效果评估\n安全往往只有 0 和 1 的概念，要么安全，要么不安全，不存在\n中间状态，因此代码安全可靠性的效果评估可通过代码泄露等安全\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n32\n事件的发生与否来评估，同时通过审计能力来保障安全事件发生时\n具备可回溯能力；\n（1）代码泄漏次数：代码仓库代码泄露事件的次数；\n（2）漏洞数量：代码仓库中发现的漏洞数量；\n（3）访问控制：是否具备精细化权限控制的能力；\n（4）审计能力：是否具备良好的审计能力；\n2.1.2.4 仓库可扩展性\n代码仓库的可扩展性是指代码仓库在面对不断增长的代码量、\n用户数量、访问频率等变化时，能够保持高效的性能，同时可以方\n便的通过软硬件来扩展功能和架构，以满足未来的需求，代码仓库\n的可扩展性对一个快速发展中的团队尤其重要。\n1．代码仓库可扩展性提升措施\n（1）分布式版本控制系统：使用支持分布式版本控制系统，比\n如 Git，可以支持代码仓库的架构可扩展性，可以将代码库分散在\n多个服务器上，从而实现横向可扩展；\n（2）集群化：使用集群化技术，如 Kubernetes 等容器编排系\n统，可以实现代码仓库的自动化部署和管理；\n（3）功能扩展：使用插件等能力，结合业务的个性化诉求，扩\n展版本控制系统的功能，满足团队的研发需求；\n2．效果评估\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n33\n代码仓库可扩展的能力可使用以下两个指标评估\n（1）架构可扩展性：代码仓库架构是否支持水平横向扩展。\n（2） 功能可扩展性：是否可以通过插件扩展版本控制系统的功\n能。\n2.1.3 构建可靠性\n构建是指在构建机上把代码、资源文件等源文件编译打包成可\n执行的程序文件的过程；在当前的持续集成/持续交付的软件开发模\n式下，若构建出现问题，则新的软件版本无法快速发布验证，软件\n质量就会受到影响，所以构建的可靠性对于软件服务的可靠性和迭\n代效率起到了重要作用；构建可靠性主要由构建效率和构建成功率\n两个方面组成；\n构建可靠性提升涉及构建工具建设与规划、业务层改造优化、\n软硬协同等多个方面，一般需要由平台 SRE、业务开发、服务 SRE\n多角色共同参与；\n2.1.3.1 构建效率\n构建效率即构建速度，取决于从构建启动到构建结束的耗时情\n况，构建耗时过长的话软件版本无法按时交付，对业务版本迭代效\n率产生了影响，构建可靠性就无从谈起。\n1．构建效率提升措施\n（1）流程自动化\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n34\n利用自动化构建工具或脚本把构建各个环节串联起来，减少各\n环节之间的等待时间；\n（2）并行化\n通过把一些构建流程从串行改成并行，优化构建流程，提升构\n建速度。\n（3）增量构建\n在构建机上执行一次构建时把构建过程中的一些临时文件、中\n间产物存起来作为构建缓存，等下一次构建时，由于一般情况下只\n有部分代码被修改了，那么没有被修改的代码就能免去构建环节，\n直接使用上次的构建缓存，这样就通过增量构建的方式减少了需要\n构建的内容，降低了构建耗时。\n针对构建机首次构建时没有缓存的问题，可以搭建构建缓存共\n享服务器（例如 UE 引擎的 DDC 服务器），一台构建机的构建缓存会\n生成到缓存共享服务器上，供其他构建机使用。\n（4）分布式编译\n相对单机有限的资源来讲，集群的力量无疑是强大的：一个人\n计算 100 道数学题，相比 100 个同样能力的人一人计算 1 道题，孰\n优孰劣不言而喻。分布式编译加速就是利用集群的资源，将单个节\n点的工作分配给一大批节点，然后再汇总结果。根据需要，资源数\n量可以近乎无限地扩充，不再受制于单机的物理架构；时间上，集\n群工作所需要的时间往往是原来的几分之一。\n（5）软硬协同\n35\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n部分构建任务比如代码预处理、资源文件处理无法通过分布式\n编译加速分发到远端，只能在构建机本地处理，此时本地构建机性\n能就成为了瓶颈，可以针对性地提升本地构建机的 CPU/内存/磁盘\nIO 性能，再结合分布式编译系统，软硬协同提升构建速度。\n（6）多进程编译\n有些编译软件默认只开启单进程编译，导致构建机硬件性能未\n得到充分利用，此时可以通过开启多进程编译来提升构建速度。\n2．效果评估\n（1）基于基线：使用构建耗时超出基线比例来评估单次构建的\n效果，每次正常构建完成后，把本次构建耗时上报上去，持续若干\n天后，我们就能得到一段时期内的多次稳定构建耗时数据，把这些\n数据求一个平均值之后，我们就得到了这一段时期的构建耗时基\n线。当一次构建耗时超出基线很多时（比如超出基线 20%），这次\n构建就可能出现了性能问题，在得到构建耗时基线以后，我们可以\n将当前构建耗时与基线进行对比来作为 SLO，比如不超过基线 10%即\n为健康；\n（2）基于阈值：按不同的业务实际情况设置不同的阈值，例如\n设置构建耗时不大于 2H 为可靠性的衡量标准；\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n36\n2.1.3.2 构建成功率\n构建成功率指指定时间内，构建成功次数占构建总次数的比\n例。构建成功率低的话，需要多次构建才能产生可交付的版本，也\n是影响构建可靠性的一个关键因素；\n1．构建成功率提升措施\n（1）保障构建环境可靠性\n构建成功率受到构建环境影响，当构建机异常时（比如缺少某\n个依赖包，连不上代码仓库，磁盘故障等），构建就会失败。我们\n需要保证构建环境的可靠性，比如通过自动化方式批量部署构建\n机，避免手动部署时遗漏某些依赖包；尽量利用云上的高可靠性网\n络、计算和存储。\n（2）PreBuild 预编译检查\n通常的构建是拉取代码后再执行后面的编译流程，这就要求开\n发编写完代码后必须提交到代码仓库再启动构建。其实可以在提交\n代码到仓库之前就把问题暴露出来，问题越早发现，修复解决的成\n本越低；提交到代码库的代码质量越高，问题越少，团队协作起来\n就越顺畅，越高效。PreBuild 就是在提交代码到仓库之前，利用本\n地 PreBuild 工具进行本地代码检查或传输代码到远端进行预编译代\n码检查，从而尽早发现问题，实现质量左移。\n2．效果评估\n（1）基于基线评估：每次构建完后将成功/失败状态进行上报，\n统计一天的构建成功率，持续若干天后，我们就能得到一段时期内\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n37\n的多天稳定构建成功率，把这些数据求一个平均值之后，我们就得\n到了这一段时期的构建成功率基线。当某天的构建成功率超出基线\n很多时（比如超出基线 20%），这次构建就可能出现了性能问题。\n在得到构建成功率基线以后，我们可以将当前构建成功率与基线进\n行对比来作为 SLO，比如不低于基线 10%即为健康。\n（2）基于阈值评估：按业务需求设置固定的成功率阈值，例如\n80%，可以取一个固定周期的所有构建进行统计分析并进行对比，例\n如以天/周/月等单位。\n2.1.4 制品可靠性\n制品为构建过程的产物，包括软件包、测试报告、应用配置文\n件等，最终提供给研发、测试等多个角色，用于下载并部署到 PC、\nconsole、移动端、服务器等多种不同的设备，从而完成发布和交\n付；\n制品的可靠性分为制品下载可靠性、制品部署可靠性、制品安\n全可靠性三个方面；\n制品可靠性提升涉及制品库工具建设与规划、安全、部署分发\n等多个方面，一般需要由平台 SRE、服务 SRE、安全人员共同参与；\n2.1.4.1 制品下载可靠性\n随着企业的快速发展，研发团队规模的扩大，为拓展全球市场\n在国内海外多地建立研发团队，制品的高效交付和管理也成为影响\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n38\n研发效率的关键，制品分发缓慢、下载困难等为代表的制品库不可\n用挑战，急需快速解决；\n1．制品下载可用性提升措施\n（1）多地分发：根据研发的不同地域分布，按需设置制品的分\n发策略，实现构建完成后多地制品分发，从而达到提升制品速率的\n目的；\n（2）P2P：制品的下载往往具备一定的峰值规律，例如早高峰会\n出现大量的集中下载，为提升并发下载的速率，可以使用 P2P 的下\n载策略，下载用户越多，速率越快；\n（3）专属工具：制品的下载使用专属下载工具，能够支持分片\n下载、 断点续传、多线程、热点缓存等特性；\n（4）镜像源加速：建设距离用户更近用户的镜像源，来提升制\n品依赖的拉取速度，减少下载中断；\n2．效果评估\n（1）下载成功率，下载成功率=1-（失败请求数/用户请求\n数），失败请求是指制品库返回的错误码为服务器内部错误码的请\n求（错误码>500）；但不包括触发频控导致的限流请求或者制品库\n升级、变更、停机而导致的失败请求。用户请求指的是 制品库服务\n器端接收到的用户发送的请求，但不包括未经身份验证、鉴权失败\n或者欠费停服状态下的请求。用户端由于黑客攻击而对制品库的请\n求，或者由于配置了跨区域复制、生命周期规则而在后端异步执行\n的请求，均不视为有效请求或失败请求\n39\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n（2）下载速度，下载速度=包大小/下载耗时，下载速度和制品\n的存储区域分布及用户所在的区域有较大的关系，可针对不同地\n域、国家可设置差异化的可用性衡量指标；\n2.1.4.2 制品部署可靠性\n在制品下载完成后，还有一项重要的下游能力，即制品的自动\n化部署，使用户实现对制品产物即时体验的能力；\n在大型业务研发过程中，多种研发角色手动安装制品产物非常\n耗时，在没有自动化部署的情况下执行诸如软件安装和升级会消耗\n大量研发人员的时间和精力，且对迭代的效率有较大的影响；\n制品部署是指通过技术手段，打通构建流水线，将构建完成的\n制品主动推送到用户多种类型终端的过程，用自动化的方式对制品\n进行分发、安装、更新，让用户以较低的时间成本获取到构建产\n物，减少临时下载制品对迭代效率的影响，尤其对于具备超大制品\n的业务类型，制品部署的可靠性尤为重要；\n1．制品部署可用性提升措施\n（1）多用途支持：为满足不同的测试验证目的，一个项目往往\n会设置多条构建流水线，例如用于逻辑、性能、开发调试等多种不\n同类型，制品部署能力需要支持到多种类型；\n（2）多平台支持：制品的分类按平台分有 iOS、安卓、PC、\nconsole 等，每种平台都多种不同类型的 OS 版本和设备型号，碎片\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n40\n化程度高，预部署能力需要良好的机型兼容适配能力，能够支持多\n种不同类型的设备和平台；\n2．效果评估\n（1）部署成功率，部署成功率=成功部署次数/总分发次数\n（2）部署人工耗时，一个完善的预分发方案应该尽量减少用户\n等待的时长，通过部署人工耗时指标来驱动减少人工参与；\n2.1.4.3 制品安全可靠性\n制品在持续构建过程中的包依赖，以及构建完成的产物（含\nDocker 镜像、npm、helm、maven 等多种不同类型的格式），可能存\n在一些安全风险，导致制品不可靠；\n1．制品安全性提升措施\n（1）漏洞扫描，制品需要经过经过漏洞、license 信息的扫描\n与分析，并对漏洞和不合规的 license 告警并输出安全合规报告，\n漏洞库需要及时更新；\n（2）设置质量红线，禁止下载未经安全扫描或者未通过安全扫\n描的制品；\n（3）访问控制，针对不同角色设置差异化的权限，防止资源泄\n露和，减少恶意盗取制品风险，权限以最小化为原则；\n（4）操作审计，提供制品库的操作审计功能，保证制品的下载\n使用等操作操作可追溯；\n2．效果评估\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n41\n（1）漏洞扫描覆盖度：扫描能力能够覆盖足够多的制品库类\n型；\n（2）漏洞扫描速度：漏洞扫描的速度和及时性，能够在漏洞产\n生后，以最快的速度发现异常；\n（3）漏洞扫描准确性：漏洞库及时更新和维护，确保制品安全\n扫描结果的准确性；\n（4）访问控制：是否具备精细化权限控制的能力；\n（5）审计能力：是否具备良好的审计能力；\n2.2 研发保障工程体系设计\n2.2.1 面向研发保障的持续集成流水线\n持续集成（CI）是现代软件开发中的关键实践，通过自动化的\n方式将代码的构建、测试、部署等环节有机结合，确保代码在每次\n提交后都能快速集成并验证，从而提高软件开发的效率和质量。面\n向研发保障的持续集成流水线设计，旨在通过一系列自动化工具和\n流程，保障工程编译，静态代码检查， 测试用例运行和部署发布。\n另外，通常情况下， 平台需要提供以下关键组件能力：\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n42\n流水线：通过可视化手段展示团队现行的研发流程，实现编\n译、测试、部署等环节的一体化管理，确保流程的顺畅与高效。\n代码检查：提供精细化的代码审查解决方案，从多个维度检测\n代码中的缺陷、安全漏洞以及规范性问题，确保产品质量得到充分\n保障。\n代码库：企业内部代码托管服务\n凭证管理：为代码库、流水线等关键服务提供多样化的凭证与\n证书管理功能，增强系统的安全性。\n环境管理：可以将企业内部的开发编译机托管至流水线，统一\n管理\n研发商店：由流水线插件和流水线模板构成，插件用于整合企\n业内部的多种第三方服务，模板则助力企业研发流程的标准化。\n编译加速：通过并行编译，编译缓存，硬件匹配优化的手段，\n显著提升构建任务的执行效率。\n制品库：通常基于分布式存储架构，具备扩展能力，其功能涵\n盖制品扫描、分发、晋级、代理、包管理等，同时提供多种依赖源\n仓库，如 generic（二进制文件）、maven、npm、pypi、oci、\ndocker、helm、composer、nuget 等，以满足不同开发需求。\n持续集成流水线的设计需要遵循以下几个关键原则：\n 自动化：尽可能将所有的构建、测试、部署步骤自动化，减少人\n为干预和错误。\n43\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n 快速反馈：确保每次代码提交后，开发者能够快速得到构建和测\n试结果，及时发现并修复问题。\n 可重复性：流水线的每一步都应该是可重复的，确保在不同环境\n下执行结果一致。\n 可扩展性：流水线设计应具备良好的扩展性，提供接口及相关的\n扩展机制，对接更多的插件，满足多样化的流水线能力需求。\n 可观测性：提供全面的监控和日志记录，便于问题定位和分析。\n在流水线可视化设计上，尽量采用 Stage/Job/Task 三层结构展\n示，方便快速定位问题。\n1.\nStage（阶段）\n 由多个 Jobs（作业）组成；\n 同一个 Stage 下的 Job 执行方式为并行，由于 Job\n之间是相互独立的，某个 Job 失败后，其它的 Job 会被运行\n到完成；\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n44\n![Image 8](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_8.jpg)\n 一个 Job 失败，则该 Stage 失败。\n2.\nJob\n可以运行在一个构建环境里，比如运行在 macOS；也可以作为\n不需要构建环境的普通任务调度编排。它有如下特性：\n由多个 Tasks(插件)组成；\n45\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n![Image 9](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_9.jpg)\n![Image 9](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_9.jpg)\n一个 Task 失败，则该 Job 失败，其余 Task 将不会运行；\nTask\n也被称为流水线插件，通常是一个单独的任务，如拉取 Git 仓\n库代码等。\nTask 必须包含在 Job 内，同一个 Job 内的 Tasks 都是从上\n往下顺序执行（启用了高级流程控制的 Task 除外）。\n2.2.2 面向研发保障的可观测设计\n面向研发保障的可观测设计旨在通过全面的监控和分析，确保\n流水线的透明性和可控性。在流水线中，构建机承受着极大的工作\n负载压力，CPU 负载接近满负荷，同时占用大量内存和磁盘空间。\n此外，由于代码和配置问题，构建任务失败的可能性较高。通过可\n观测性工具，团队能够实时监控流水线状态，快速定位和解决问\n题。\n要做好流水线的可观测性， 要从以下几个维度进行考虑：\n1. 构建机的监控\nCPU 和内存使用率：监控构建机的 CPU 和内存使用情况，确保\n资源分配合理，防止资源耗尽导致构建失败。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n46\nGPU 使用率：对使用 GPU 加速的构建任务，监控 GPU 的使用情\n况，包括利用率、温度和显存占用，以确保其高效运行。\n磁盘空间：定期检查磁盘使用情况，清理不必要的文件，确保\n有足够的空间进行构建。特别是游戏制品巨大\n网络流量：监控网络流量，确保构建机与其他系统的通信顺\n畅，避免因网络问题导致的构建延迟。\n2. 流水线监控\n构建状态：实时监控每个构建任务的状态，包括成功、失败、\n进行中等，帮助快速识别问题。\n任务队列：监控流水线中的任务队列，优化任务调度，防止任\n务积压。\n执行时间：分析各个阶段的执行时间，识别性能瓶颈并进行优\n化。\n3. 日志监控\n错误和警告：重点关注日志中的错误和警告信息，及时采取措\n施解决问题。\n历史日志分析：通过分析历史日志，识别常见问题和趋势，改\n进构建流程。\n4. Trace 跟踪\n47\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n构建的过程基本上等同于一条 Trace ， 通过对构建过程进行\nOpenTelemetry 等链路跟踪协议后， 进行上传， 可以实现更细粒\n度的构建成功跟踪。如下图，通过上报到 Jaeger 后， 可以实现每\n个编译步骤的消耗时长的可视化。\n5. 告警触达\n该平台将指标、仪表盘以及告警信息推送至研发团队，使研发\n人员能够更全面地掌握持续集成环节中出现的问题，从而进一步释\n放系统可靠性工程师（SRE）的人力资源，并提高研发效率，降低沟\n通成本。\n2.2.3 面向研发保障的操作调度操作平台\n操作调度平台在研发保障过程中发挥了至关重要的作用，确保\n各项操作任务能够高效、准确地执行，并减少人为干预带来的风\n险。该平台主要负责以下几个关键工作领域：\n1. CD 发布管理\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n48\n![Image 11](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_11.jpg)\n在持续交付（Continuous Delivery, CD）过程中，操作调度平\n台承担着发布管理的核心职责。它不仅能确保从代码提交到生产环\n境的全过程自动化和高效运作，还能够通过智能调度来优化资源使\n用和任务执行的顺序。\n自动化流程编排：操作调度平台通过自动化流程编排，将代码\n提交后的各项任务（如编译、测试、部署等）整合到一个无缝的流\n程中，并自动调度这些任务的执行顺序，以避免资源冲突或执行瓶\n颈。通过与 CI/CD 工具链的深度集成，平台能够在任务之间进行动\n态调度，确保每个流程步骤都在最佳时间点执行。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n49\n![Image 12](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_12.jpg)\n智能调度算法：平台利用智能调度算法，根据任务的优先级、\n依赖关系和资源可用性，动态调整任务的执行顺序和分配的计算资\n源。这种调度策略不仅能够优化资源利用率，还能提高部署效率，\n减少因资源不足或冲突导致的延迟。\n2. 批量化操作\n批量操作是操作调度平台的另一重要功能，它支持对大规模操\n作任务的统一管理和调度，极大地提高了研发团队处理日常运维任\n务的效率。\n统一任务调度：操作调度平台能够将大量重复性、标准化的操\n作任务（如批量更新配置、批量重启服务等）整合到一套自动化流\n程中，并通过统一的调度机制进行管理。\n并行执行与任务分片：为了提高批量操作的效率，平台支持并\n行执行和任务分片技术。它能够将大规模任务分解为多个小任务，\n并分配到不同的计算节点并行执行，从而显著缩短任务的整体执行\n时间。此外，平台还具备任务依赖管理功能，确保各子任务按顺序\n正确执行。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n50\n动态资源分配：批量操作过程中，平台能够动态分配计算资\n源，根据任务的实时需求调整资源分配策略，以确保在高峰期也能\n保持高效执行。\n2.2.4 面向研发保障的 ITSM 平台\nIT 服务管理平台（IT Service Management, ITSM）在研发保\n障中起到了至关重要的作用，为研发团队提供了统一、标准化的管\n理工具和流程，确保研发过程中的各类服务请求、事件管理、问题\n解决、变更控制等工作能够高效、有序地进行。\n2.2.5 面向研发保障的容器平台\n容器平台为研发团队提供了一个高效、灵活、可扩展的环境，\n以便快速部署、测试、和迭代应用。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n51\n![Image 13](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_13.jpg)\n为了有效支持研发团队的工作流程，容器平台必须具备一系列\n专门设计的功能，确保研发过程的高效和稳定。\n自动化 CI/CD 管道集成：容器平台与 CI/CD 工具链无缝集成，\n支持自动化构建、测试、和部署。通过 Jenkins、GitLab CI 等工\n具，容器平台能够在代码提交后自动触发构建和部署任务，大大缩\n短了研发周期。\n环境隔离与多租户支持：通过命名空间和 RBAC（Role-Based\nAccess Control）等机制，容器平台可以实现环境隔离，支持多个\n团队和项目的并行开发。这种隔离不仅限于资源使用，还包括网络\n和安全配置，确保不同团队之间的独立性和安全性。\n观测性平台对接：容器平台需要能与观测平台实现无缝的对\n接， 将运行于容器的指标， 日志， 链路上报的观测平台中， 并\n能实现监控告警。\n敏捷发布与回滚机制：容器平台支持蓝绿部署、金丝雀发布等\n敏捷发布策略，能够在不影响整体服务的前提下逐步发布新版本并\n监控其表现。一旦发现问题，平台可以快速回滚到之前的稳定版\n本，减少对业务的影响。\n测试环境治理：针对实际研发工作环节中，测试环境搭建流程\n过长，测试资源无法充分利用和测试设备日常维护人力成本高的问\n题，基于容器平台，以 k8s 工作负载的方式部署运行管理多个测试\n环境。具体架构如下\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n52\n2.2.6 面向研发保障的编译加速平台\n编译构建是项目开发和发布过程中的重要环节，同时也是非常\n耗时的环境，有些项目执行一次完整的构建需要几十分钟甚至几个\n小时，各种编译构建加速工具都能在一定程度上减小构建时长。\n整体架构如下：\n 编译加速工具包运行于用户构建机，为构建机上的编\n译任务接入加速服务\n 分布式编译服务为集中式部署，负责接受编译加速工\n具包加速请求，调度加速资源，并负责加速任务过程全生命周\n期管理\n 分布式资源集群通过分布式编译服务引擎服务统一动\n态调度，直接为用户编译提供分布式加速\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n53\n![Image 14](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_14.jpg)\n分布式加速底层通过 disttask 分布式任务引擎实现，disttask\n各模块功能介绍如下：\n remoter worker 运行在分布式资源集群中中，负责接收，执行\n和返回分布式任务\n local server 运行在构建机上，实现分布式任务底层基础功\n能，并可扩展不同应用场景的分布式任务实现\n disttask_executor 通用任务执行器，接管实际编译中的编译命\n令(如 gcc 命令，clang 命令)，是构建工具和分布式基础服务之\n间的桥梁\n 基于 disttask 提供的接口，根据实际场景需要，实现独立的构\n建工具\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n54\n![Image 15](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_15.jpg)\n2.3 研发保障案例\n2.3.1 腾讯游戏全球研发保障实践\nSRE Elite 精选原因\n这是一个完整的游戏行业研发保障案例。面对游戏研发中的复\n杂研发管线、大文件版本管理、冗长的构建过程和频繁的更新需求\n等挑战，SRE 团队通过稳定性保障、平台工具建设、以及与业务开\n发团队的有效分工，实现了高效的研发保障。\n此案例覆盖了研发保障的多个关键模块，在代码可靠性，代码\n仓库可靠性、制品分发、以及构建加速等多个方面进行了优化，显\n著提升了代码提交和构建的成功率，并有效解决了代码库卡顿和文\n件分发效率低等问题。相关的优化内容非常的详尽细节， 具有很强\n的实践性，且大部分关键组件提供了开源的实现案例，非常值得参\n考。\n（一）背景及设计原则\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n55\n大型游戏研发对研发管线有独特的要求\n伴随着腾讯游戏多元化的诉求日趋增长，越来越多的游戏开发\n团队开始尝试多地共研模式，这在工程版本控制、编译构建、制品\n共享等方面都提出了新的挑战。\n游戏行业的研发具有以下独特特点和挑战：\n1. 复杂且研发管线：用户提交的不仅是源代码，还可能包含\n体积达数百兆的单个素材文件，这些文件需要进行版本管理，而且\n还需要对原始素材文件为不同终端进行渲染，耗时巨大，出包过程\n可能长达 5-6 小时。而如果中途构建失败，第二天项目进度就会受\n到影响。\n2. 版本库性能：版本库中包含大量大文件，整个代码库的体\n积可能超过上 PB，而且随着游戏生命周期的延续，代码库只会越来\n越庞大。以此未经优化前，用户在访问或修改版本库里头的资源时\n容易卡住，而游戏团队的规模可能达到上百人，人力成本极其高\n昂，如果卡住一个小时损失就会非常巨大。\n3. 项目团队协作及代码质量问题：目前游戏的制作已经进入\n工业化生产的阶段，每个游戏项目可能都有几十人甚至上百人的参\n加，如何解决由于参与人数众多而带来的代码冲突以及代码质量不\n统一问题。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n56\n4. 业务制品构建耗时长，严重影响协同效率和敏捷开发。对于\n普通互联网应用安装包普遍在 500M 以内，单次构建在 30 分钟以\n内，游戏安装包和资源文件加起来一般可达 5G-20G 以上，客户端包\n单次构建耗时需要 2-5H 左右；\n5.激进的发布时间：游戏行业竞争极其激烈，游戏内部经常衍\n生出新的玩法和素材，用户期望极高，玩家口味变化极快，前期宣\n发费用极其高昂，项目必须如期上线才能减少前面投入的风险。\n6.频繁的更新：根据用户反馈，游戏需要频繁出包进行调整，\n并引入新玩法、新皮肤和新角色，\n7.制品分发效率及安全：现在的游戏动则几十到上百 GB 的包，\n如何进行快速，稳定地分发，送到大量的测试及渠道用户。如何保\n障制品的安全。\n8.多方协作带来的安全问题：游戏行业已经从从小作坊，演变\n成类似于好莱坞式工作室的分工协作模式，大量的资源需要通过外\n包的形式进行分包协作。由此带来安全问题非常突出，不少的竞争\n对手或者说相关的产业都会盯上我们的整体的研发流程，在游戏新\n版本的研发过程中，如果一旦出现了资源的泄露，玩法的泄露，数\n值的泄露，有可能就会导致竞争对手或者黑产的狙击，很有可能导\n致版本需要重做， 数百人月的工作量付之东流。如牵涉到核心资\n产， 如核心代码， 核心引擎等，甚至会出现私服的情况，影响整\n个游戏的未来的生存。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n57\n举个例子，某日凌晨 1 时许，由于机房故障，部分流水线的构\n建机下线，我们本以为这么晚应该不会有用户进行构建，本决定第\n二天再修复。结果，凌晨 2 点和 3 点就有用户在公司论坛进行投\n诉，据了解，因为代码改动直接到了当天凌晨 1 点多，因此他们的\n构建时间只能选择在凌晨 2 点，才能满足版本在当天进行提测发版\n的进度要求。此事让我们更深刻理解到研发管线的可靠性对业务的\n重要价值。\n总之，解决以上问题，做好研发保障，保障研发管线的高效安\n全运行，价值巨大，挑战巨大。\nSRE 团队负责研发保障是一个合适的选择\n研发阶段的可靠性是指，以 SRE 理论驱动研发的可用性建设，\n提升研发管线的工业化水平，保障版本能够按正常周期迭代，从而\n实现高质量持续交付有效价值的目标\n从软件工程的角度：\n SRE 的稳定性保障工作，本质上是软件工程的一部分，也就是运\n维左移，从源头保障软件的质量\n从 SRE 专业能力的角度：\n 利用 SRE 团队既有平台能力又有服务能力的特点，从平台和服务\n两个地方切入，建立 SRE 团队主导的工具平台和服务体系，让研\n发团队在平台和服务上需求都得到最大化的满足。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n58\n 服务上，SRE 团队做研发服务，可以复用现有的运维支持体系，\n与研发相比， SRE 有高度的业务可用性意识和能力， 能很好的\n保障业务研发工具链的可用性，防止因为工具链的不可用或性能\n低下，影响业务团队研发效率。\n 工具上，SRE 团队可以复用，开发，并扩展在 CO /CD 领域的工\n具去进行研发保障，例如，利用可观测的平台， 对研发工具链\n的健康状态以及使用体验进行观测，度量研发工具链上下游健康\n状态，故障时快速定位根因，同时提供成熟的可观测落地方案，\n标准化实施，降低建设成本\n从组织分工上的角度：\n SRE 团队负责研发保障，业务开发负责产品功能开发，对于企业\n来说，可以发挥最大的投入产出比。SRE 可以更体系化和规模化\n进行全面保障。业务开发的本质逻辑是更加聚焦快速开发迭代版\n本，完成产品功能开发。理想状态下，业务开发无需关注研发平\n台上下游所有组件的稳定性。\n 从职业发展来说，SRE 工程师在 SRE 组织中，有明确的成长路\n径和岗位通道，且通过从 CO/CD 扩展到 CI 域，扩展了职业发展\n方向，并且与相关能力设置在业务开发团队相比， 更具有稳定\n性和成长性。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n59\n简单来说，SRE 团队是既有服务意识，又有平台及产品的建设\n能力，从软件工程的角度整体保障角度，完成了业务研发不擅长但\n是有迫切需求的工作。因此， 我们认为由 SRE 工程师去从事研发\n保障的工作，是一个高 ROI ，合乎组织利益及各方干系人利益，能\n提升研发效率，可持续的解决方案。\n本案例最后总结部分，会有我们业务研发同学对 SRE 从事专业\n度的认可和反馈。\n在此基础上，SRE 切入研发保障，需要注意以下三点：\n稳定高效：需要 SRE 团队在服务上有韧性，和研发团队建立信\n任。\n低成本定制：SRE 团队能够低成本的为每一个业务定制研发流\n水线方案，定制个性化需求，例如兼容多种代码库等这就要求我们\n的方案必须是 PaaS 模式，也就是平台工程的效果。\n安全可靠：一方面是研发团队对代码安全的要求，例如外包开\n发团队的代码安全。我们通过离岸云研发模式解决另一方是研发环\n境的安全性，避免代码泄露事件发生。\n平台能力和服务能力方面，做到稳定高效，低成本定制，安全\n可靠，让用户对 SRE 团队提供的研发服务产生依赖，用户用的越深\n入，与 SRE 团队的绑定就越紧密，SRE 团队的研发服务就会可持\n续，从而建立长期服务关系。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n60\n（二）体系设计及关键流程\nSRE 团队提供的全托管研发保障体系，涵盖从代码提交、代码\n仓库管理、构建流程到制品分发的全过程。通过代码分析、仓库安\n全管理、编译加速、制品共享等多项服务，确保了研发过程的高效\n性和安全性。大部分情况下，研发只需要往平台提交代码的后，后\n继的环节将不需要进行干预，即可高效获取到相应的制品。\n图中还包括由 SRE 团队研发并提供的支持的 PaaS 平台，SRE 团\n队利用相关的平台能力，提供持续集成、观测、批量操作、ITS 管\n理、计算和容器平台等功能，进一步优化研发环境。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n61\n![Image 16](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_16.jpg)\n整体个体系强调 SRE 团队在提升研发效率、保障流程稳定性和\n安全性中的关键作用。\n备注：案例中可能会提及到各个平台的简称， 此处进行统一进\n行备注说明：\n●蓝盾：蓝鲸持续集成平台的简称，开源地址\nhttps://github.com/TencentBlueKing/bk-ci\n●标准运维：蓝鲸智云标准运维的简称，开源地址\nhttps://github.com/TencentBlueKing/bk-sops\n●ITSM：蓝鲸流程服务的简称，开源地址\nhttps://github.com/TencentBlueKing/bk-itsm\n●容器平台：蓝鲸容器管理平台简称，开源地址\nhttps://github.com/TencentBlueKing/bk-bcs\n●BKTurbo：蓝鲸编译加速平台简称，开源地址\nhttps://github.com/TencentBlueKing/bk-turbo\n●CodeCC 代码检查：CodeCC 插件(腾讯代码分析插件），开源\n地址 https://github.com/TencentBlueKing/ci-\nCodeCCCheckAtom\n●制品库：蓝鲸制品库简称，开源地址\nhttps://github.com/TencentBlueKing/bk-repo\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n62\n（1）代码可靠性研发研发保障实践\n某游戏客户端通过质量红线保障各阶段代码质量\n在某游戏客户端业务研发过程中，我们经常会有类似的困惑：\n 确立了团队的编码规范，但还是有不符合规范的代码被合入；\n 拥有自己的转测试标准，但是只能人工跟进，不能落实到自动化\n流程中；\n 在版本的后期，Bug 数居高不下。但很多问题是可以在前期通过\n代码检查和单元测试发现的。\n为了解决上述问题，借鉴丰田精益生产的思想，腾讯打造了质\n量红线 Gate 服务。\n质量红线的思想起源于丰田精益生产的立即暂停系统（stop-\nthe-line Andon），又称为安灯（Andon）系统。当车间生产流水线\n上的员工遇到麻烦，立即拉一下信号灯，班组长就会立即跑过来帮\n助解决，其生产流水线也会停止直到问题解决。这样能够尽早暴露\n问题，解决问题，而不是把问题流到生产汽车的后续步骤。\n解决方案\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n63\n质量红线是指通过设置质量标准，控制流水线的行为，使得每\n一阶段的出口质量都必须符合质量标准的一种服务。\n设计思想\n设计思想如下\n1、独立成质量红线服务，基于关键点控制质量\n从用户使用场景出发，将开发、部署测试环境、部署生产环境\n等相关的插件（共 10 个）选出作为关键控制点。这些控制点插件在\n流水线中可以非常灵活地编排，但只要质量红线设置了标准，就必\n须质量达标控制点才能执行通过。\n2、指标灵活可扩展\n设计了统一的指标定义规范和指标灵活可扩展的机制。主要包\n括系统插件指标（主要是 CodeCC 代码检查）、脚本任务指标、 研\n发商店插件指标；\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n64\n![Image 17](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_17.jpg)\n3、支持为代码合入设立红线\n支持设置质量要求，并通过流水线 Git/Github 事件触发机制与\n工蜂/Github 串联起来。以工蜂为例，当用户在工蜂发起 MR 时，会\n通过 Merge Request hook 触\n\n1.使用统一账号系统，提供统一鉴权；\n2.元数据（含权限表、用户表、trigger 等）自动备份，一键\n恢复；\n3.集群化部署，主备容灾，使用云盘避免机器故障导致数据丢\n失；\n4.代码拉取、提交、删除、变更记录接入公司审计系统；\n5.人员离职、组织架构变更，邮件提醒权限删除；\n6.禁止供应商 office 网络连接 P4，只能在云桌面内部访问；\n7.p4auth 服务器单独架设，commit 异常，切换 replica；\n网址：SRE-Elite.com\n\n高可用是运维管理的一条底线保障要求，运维主要工作是消灭\n单点风险，提升系统韧性，比如数据库中提到的主备、主从、分布\n式，数据中心的两地三中心、分布式多活，以及将一个应用系统同\n一个服务组件部署在多个数据中心机房、不同物理机的多个虚拟机\n上、为应用的负载均衡提供网络硬件或软件负载均衡器、提供具备\n高可用架构消息中间件等 PaaS 云服务等。为了更好的推进评估工\n作，SRE 需要提前提供架构高可用的规范，制定通用组件、信息系\n统架构高可用参考模式，将高可用要求更早的落地在系统设计过程\n中。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n180\n故障恢复可借鉴最佳实践、具体信息系统的特点等，制定相应\n的故障恢复能力要求。在应用系统层面，需关注应用拆分、服务或\n系统交互解耦、服务无状态、减少总线节点服务依赖、增加异步访\n问机制、多层次的缓存、数据库优化、限流与削峰机制、基础设计\n快速扩容等。在基础设施层面，可恢复性的基础设施环境需能够有\n效地从自然灾害或人为灾难中恢复，即考虑到各种风险，包括自然\n灾害、技术灾难、人为错误或网强行攻击等，并采取预防措施以减\n少潜在的损失，至少应包括备份、冗余、快速恢复服务和关键信息\n系统的保护。\n技术架构的可扩展性是指在不影响现有系统功能和性能的前提\n下，系统能够扩展或增强其功能的能力。通常涉及对系统设计、架\n构和模块化的考虑，以便于未来的扩展和升级。这包括将系统拆分\n为多个独立的子系统或模块，每个模块负责特定的功能和业务逻\n辑，降低整个系统的复杂度与模块间的耦合度，提高扩展性；支持\n横向和纵向的扩展能力，通过增加服务器数量或提高每个服务器的\n处理能力，来提升整个系统的处理能力，或通过升级单个服务器或\n组件的硬件来提高其处理能力；系统支持弹性伸缩，即根据系统的\n负载情况自动调整计算和存储资源，以实现系统的动态扩展和缩\n减；减少对总线节点服务的依赖，将多节点组成的逻辑交互改为端\n对端访问方式，以减少交易影响因素，避免单节点性能问题影响其\n他应用系统；增加异步访问机制，因为同步机制在性能出现问题\n时，会在短时间内耗尽最大连接数，即使这个最大并发数是正常情\n网址：SRE-Elite.com\n\n高可用是运维管理的一条底线保障要求，运维主要工作是消灭\n单点风险，提升系统韧性，比如数据库中提到的主备、主从、分布\n式，数据中心的两地三中心、分布式多活，以及将一个应用系统同\n一个服务组件部署在多个数据中心机房、不同物理机的多个虚拟机\n上、为应用的负载均衡提供网络硬件或软件负载均衡器、提供具备\n高可用架构消息中间件等 PaaS 云服务等。为了更好的推进评估工\n作，SRE 需要提前提供架构高可用的规范，制定通用组件、信息系\n统架构高可用参考模式，将高可用要求更早的落地在系统设计过程\n中。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n180\n故障恢复可借鉴最佳实践、具体信息系统的特点等，制定相应\n的故障恢复能力要求。在应用系统层面，需关注应用拆分、服务或\n系统交互解耦、服务无状态、减少总线节点服务依赖、增加异步访\n问机制、多层次的缓存、数据库优化、限流与削峰机制、基础设计\n快速扩容等。在基础设施层面，可恢复性的基础设施环境需能够有\n效地从自然灾害或人为灾难中恢复，即考虑到各种风险，包括自然\n灾害、技术灾难、人为错误或网强行攻击等，并采取预防措施以减\n少潜在的损失，至少应包括备份、冗余、快速恢复服务和关键信息\n系统的保护。\n技术架构的可扩展性是指在不影响现有系统功能和性能的前提\n下，系统能够扩展或增强其功能的能力。通常涉及对系统设计、架\n构和模块化的考虑，以便于未来的扩展和升级。这包括将系统拆分\n为多个独立的子系统或模块，每个模块负责特定的功能和业务逻\n辑，降低整个系统的复杂度与模块间的耦合度，提高扩展性；支持\n横向和纵向的扩展能力，通过增加服务器数量或提高每个服务器的\n处理能力，来提升整个系统的处理能力，或通过升级单个服务器或\n组件的硬件来提高其处理能力；系统支持弹性伸缩，即根据系统的\n负载情况自动调整计算和存储资源，以实现系统的动态扩展和缩\n减；减少对总线节点服务的依赖，将多节点组成的逻辑交互改为端\n对端访问方式，以减少交易影响因素，避免单节点性能问题影响其\n他应用系统；增加异步访问机制，因为同步机制在性能出现问题\n时，会在短时间内耗尽最大连接数，即使这个最大并发数是正常情\n网址：SRE-Elite.com\n\n基于上述背景，IT 运维部一直在思考如何建设一个变更一体化\n平台，抽象出四个“一体化”的思路：自研和外购一体化、传统和\n云原生一体化、测试和生产一体化、敏捷和安全一体化。\n（1）自研和外购一体化\n外购业务系统是由厂商维护的，厂商提供的就是一个制品包，\n因此需要支持厂商运维人员在平台维护程序包，并完成发布动作。\n变更一体化平台支持运维人员在界面上手动上传程序包，存储\n在共享存储服务中。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n327\n![Image 200](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_200.jpg)\n为了避免存储堆积，可以自定义设置存储清理策略：\n 永不清理\n 按天数清理\n 按版本清理\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n328\n![Image 201](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_201.jpg)\n![Image 201](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_201.jpg)\n在发布任务中，会将程序包存储的源服务器 IP 和源地址，以及\n分发的目标 IP 和目标路径，作为参数变量传给标准运维流程，这样\n可以通过作业平台的原子实现，文件的快速分发。\n标准运维中引用快速分发文件的原子，利用 GSE 的大文件分发能\n力，可以快速自动将文件分发到目标机器上。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n329\n![Image 203](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_203.jpg)\n自研业务系统是开发团队维护，通过流水线打包之后，自动入库\n到标准的制品仓库中，在测试环境和生产环境中都是存储在制品仓\n库中。为了维护制品包的一致性，变更一体化平台支持通过制品库\n插件，对接制品仓库同步程序包信息。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n330\n![Image 204](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_204.jpg)\n上传制品库插件\n针对这种类型，变更平台不会再存一份程序包，因此是调用制品\n库接口获取程序包的名称、版本、MD5、下载地址，用于做程序包分\n发。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n331\n![Image 205](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_205.jpg)\n![Image 205](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_205.jpg)\n![Image 205](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_205.jpg)\n考虑到直接开通所有主机到制品仓库的网络策略，存在较大的风\n险敞口，以及各个云区域到制品仓库的网络速度限制，不会在目标\n机器上直接执行 curl 命令下载程序包。\nSRE 同学编写了自定义的文件分发原子，用一台代理机器，和制\n品仓库在同一个云区域，先执行 curl 下载命令，将程序包从制品仓\n库下载到本地，再通过作业平台的文件分发功能，实现大文件的快\n速分发。\n通过兼容设计，将两种类型的制品包统一管理起来，用同一套发\n布逻辑，完成制品包的分发和部署，实现自研系统和外购系统的一\n体化。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n332\n![Image 208](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_208.jpg)\n（2）传统和云原生的一体化\n公司的传统应用部署在主机上，发布时需要登录主机操作，而\n云原生应用部署在 K8s 集群上，发布时需要执行 kubectl 命令完\n成。另外部分工具的主机发布任务和容器发布任务存在割裂，无法\n统一执行。而应用发布中心支持一个任务同时兼容主机发布和容器\n发布，实现一次审批一次编排。\n针对主机场景，使用管控平台纳管所有主机，标准运维编排所\n有主机发布任务。并且针对同类型的业务系统，规范建议复用相同\n的标准运维流程，无需每个业务团队自己建设。通用化规范化发布\n流程建议：\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n333\n![Image 209](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_209.jpg)\n![Image 209](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_209.jpg)\n(1)分发制品包：将具体的制品包名称、分发 IP、分发路径设\n置为变量，由业务系统在发布时传入，这样所有业务系统的制品分\n发复用这一个规范分发节点\n(2)备份程序：完成原有位置的程序备份\n(3)屏蔽告警策略：传入目标主机 IP，或者屏蔽策略 ID，停服\n务前自动完成告警消息屏蔽。\n由于变更时存在服务启停，变更期间必然产生大量无效告警，\n因此需要将变更目标主机的进场告警屏蔽。此操作也是引用一个通\n用的监控平台原子。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n334\n![Image 211](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_211.jpg)\n(4)停止服务：执行停服务脚本\n(5)替换程序包：替换程序包\n(6)重启服务：执行启服务脚本\n(7)端口检查：检查进程端口是否启动\n(8)地址检查：检查业务访问地址访问是否正常，编排一个通用\n检查脚本，具体 URL 通过参数变量传入，多业务系统复用此脚本\n(9)解除告警屏蔽：变更完成之后，自动解除告警屏蔽\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n335\n![Image 212](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_212.jpg)\n![Image 212](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_212.jpg)\n(10)发送通知\n针对云原生的应用，变更平台支持编排容器应用的 yaml 文件，\n提供两种编排方式，一种是原生管理方案，支持运维人员直接导入\nyaml 文件进行管理；一种是将 yaml 文件中的关键参数抽取成表单\n参数，运维人员只需要下拉选择即可，降低容器应用的编排门槛。\n做容器发布时需要调用 K8S 接口，变更平台通过 yaml 插件集成\n接口实现容器发布。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n336\n![Image 214](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_214.jpg)\n因此变更一体化平台，支持将传统主机发布节点，和云原生发布\n节点，编排到同一个发布任务中，实现“传统和云原生的一体\n化”。\n此外考虑到还存在数据库的 SQL 更新、纯配置更新等不同场景，\n变更平台在调度引擎时，单独设计的插件模块，支持通过可扩展的\n插件去接入更多发布类型。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n337\n![Image 215](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_215.jpg)\n![Image 215](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_215.jpg)\n（3）测试环境和生产环境的一体化\n为了提高生产环境的发布成功率，要求生产环境的发布操作，\n尽可能 1 比 1 复刻测试环境的发布操作，达到测试环境和生产环境\n的一体化。\nIT 运维部的 SRE 专家将发布操作拆解成三个环节，每个环节都\n需要保证测试环境和生产环境的一致性。\n环节一：应用拓扑\n在测试环境一般只会有一套集群，而生产环境两地三中心，至\n少存在三套集群，一般会认为测试和生产的环境没办法保持一致，\n从而忽略测试环境的维护。但实际上可以在应用拓扑层级保持一\n致，并且按照同一套规范去维护。\n公司在测试环境、预发环境、生产环境都使用“业务-子系统-\n集群-模块”四层拓扑架构，减少架构层面的疏漏。\n环节二：制品管理\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n338\n![Image 217](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_217.jpg)\n在测试环境和生产环境都部署独立制品仓库，开通从测试环境仓\n库到生产环境仓库的单向定点网络，并创建定期制品同步策略。\n开发人员在 CI 流水线阶段完成构建打包之后，就会自动入库，\n将版本包存储到制品仓库中；在测试人员完成测试后，自动触发制\n品晋级任务，自动将制品从测试环境的制品仓库同步到生产环境的\n制品仓库，用于生产变更。\n这样确保生产环境要发布的制品包，是经过测试，并且和测试\n环境的版本是一致的，减少运维人员在过程中的手动下载上传干\n预，全流程自动化流转。\n环节三：发布流程管理\n发布流程时生产环境变更中的最核心的部分，更要求生产环境\n的发布流程，是在测试环境中完整执行过的。且为了降低人工重新\n配置的复杂度以及出错概率，变更平台中的发布任务模板和标准运\n维执行流程，都是支持从测试环境导出，直接导入到生产环境，保\n证发布流程的一致性。\n变更一体化平台从应用拓扑、制品管理、发布流程管理三个维\n度，实现测试环境和生产环境的一致性，提高生产环境变更的稳定\n性。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n339\n（4）敏捷和安全的一体化\n在测试环境，开发人员通过 CI 流水线做持续集成和持续部署，\n实现快速开发迭代，因此对变更平台的要求是：能够作为 CD 节点完\n美到 CI 流水线中，实现应用的快速部署。\n变更平台开放了丰富的 OpenAPI 去满足 CI 流水线的要求：\nsearch_template 接口支持传入业务系统查询发布任务模板\ncreate_task 接口支持传入任务模板 id 创建发布任务\noperate_task 支持操作发布任务，包括执行、暂停、灰度、停\n止\n在生产环境，最重要的要求是稳定安全，因此需要对业务变更\n操作进行更多管控。一方面是在重要时间段内不允许变更，比如证\n券交易期间，重大会议期间；另外一方面所有变更操作需要和工单\n系统对接，运维领导审批通过之后才能执行发布。\n340\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n![Image 218](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_218.jpg)\n创建业务保障策略，在重大会议期间、节假日期间，从技术上\n限制运维人员对生产环境进行变更操作。目前大部分金融企业都是\n采用管理规范进行约束，在此期间不允许进行变更，但无法落地到\n技术层面进行强管控，还是存在业务团队冒险变更的风险。深层原\n因是，没有一体化的变更平台，每个业务团队使用各自的变更工具\n进行操作，无从限制。\n业务保障策略支持灵活配置满足不同场景需求：\n 支持设置策略状态，默认启用/停用\n 支持限制超管\n 支持选择生效业务，不同业务的安全级别不同，不同时间管\n控要求不一样，C 端业务强管控，内部系统弱管控；以业务\n系统级别管控，而不是具体 IP 维度管控，降低运维复杂度\n 支持自定义设置生效时间，日常证券开市时间是周期性周\n一-周五，节假日是固定时间段\n通过工单插件对接现有的变更工单系统，实现生产变更操作的\n审批。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n341\n![Image 219](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_219.jpg)\n在测试环境，通过 OpenAPI 集成到 DevOps 流水线中，实现快速\n发布，达到“敏捷”要求；在生产环境通过业务保障策略和工单审\n批策略，实现发布管控，达到“安全”要求。\n(三) 总结及展望\n变更一体化平台建设是一套长期动态的系统工程，随着应用系\n统及周边配套平台联动，还有很多工作在规划中，有很多能力在持\n续完善，例如度量、复盘、规范化等等。另外一些源自互联网的新\n方案例如 argoCD 等相对于传统的 K8s 部署方式集成度更高，可以\n探索其与合规跟质量的结合点。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n342\n5.1 故障体系\n5.1.1 故障定义\n故障的定义如下：在日常运营中，无论任何原因导致我们的服\n务中断、服务品质下降或用户服务体验下降，都被视为故障。但不\n包括用户环境活自身操作引起的问题。\n5.1.2 故障等级\n故障等级是衡量故障影响度的标准，数字越小级别越高。通常\n业界是以 P0 P1 P2 P3 P4 P5 等作为序列， P0 为影响度最高的故\n障（也有部分公司以 P1 作为最高级别）。\n影响度参考指标包括客户投诉量、资金损失金额、受影响用户\n数，故障时长，业务重要性等。\n通常情况下， 故障等级由 GOC、业务开发、质量团队、业务\nSRE 和业务稳定性等部门共同制定。而且相关故障等级需要提交业\n务负责人审批，确保业务方对相关故障级别标准的认可。并定期由\nGOC 统一审核故障定义和等级，并公示，确保大家按统一标准执\n行。\n故障等级的作用，我们可以基于这些定义，开展可观测性建设\n（配置针对性的告警体系以及预案）和平台能力构建（包括而不仅限\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n350\nSRE 实践白皮书（2024 年）\n于可用性架构设计及评估， 压测，混沌演练， 容灾演练），目的\n是提前识别潜在问题，进行预防和保障。\n在故障发生后，需要对故障进行评估，并采用就高原则， 如果\n一个案例符合多条故障定义，通常以最高等级为准。\n5.1.3 故障序列\n高稳定性团队可能会面临因非技术问题（如业务运营方配置失\n误，网络运营商的抖动或其他外部因素）引起的故障，因此需要明\n确技术故障与外部原因之间的区别，因此需要引入故障序列。\n故障序列是根据故障的根本原因和实际影响进行分类的方法，\n区分“故障根因”和“实际影响”。\n在定义故障序列时，主要考虑以下几点：\n技术责任：判断故障是否由技术原因引起，如是，则可能扣减\n故障预算。我们关注 P1 到 P5 等级的故障，因为它们通常涉及技\n术责任并影响业务。\n故障记录与配额：评估年度稳定性需考虑故障记录和配额。许\n多公司限制每年故障次数，1超出后可能影响绩效。因此，非技术原\n因的故障，如业务操作失误，不扣减预算。例如，外部原因的故障\n标记为“W”，业务操作引起的标记为“S”。公司可根据这些分类\n制定定义更明确相关的稳定性指标。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n351\nSRE 实践白皮书（2024 年）\n5.1.4 关键业务场景\n组织有众多业务，究竟该优先保障哪些？我们应聚焦于最重要\n的业务，而非面面俱到。因此，我们定义了关键服务接口和结果\n点，即 关键业务场景， 也称 GOC（Global Operation Center）场\n景，也有公司叫 NOC (Network Operation Center） )。只要确保\n这些 GOC 场景的稳定，组织就不会出现重大问题。\n关键业务场景通过以下三个领域定义：\n 场景描述：明确需要保障的业务场景及其功能。\n 场景影响评价：设定衡量指标（如成功量、成功率、响应时间）\n来评估场景的稳定性和性能。可以理解为 SLI /SLO 体系的构\n建。\n 故障等级：明确场景故障时可能涉及的最高等级。重要业务需重\n点保障以避免高等级故障。\n实例说明\n以登录功能为例：\n免密登录：使用成功量作为衡量指标，若成功量下降，则可能\n触发相应故障等级。\n账号密码登录：使用成功率进行衡量。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n352\nSRE 实践白皮书（2024 年）\nGOC 场景定义的目的：\n通过定义 GOC 场景，聚焦关键服务接口和结果点，驱动和确保\n公司整体稳定性。\n围绕 GOC 场景，开展容量评估、压测、限流、监控、变更管理\n等各项工作。\n 业务链路： 清晰了解所保障业务的核心目标和完整的业务链\n路。明确稳定性保障团队，当问题发生时，能够迅速召集相关人\n员共同解决。\n 容量模型：从接入层到中间件，再到下游数据库，全面了解系统\n架构；建立性能基线， 明确资源成本；描绘流量模型，识别不\n同时间段的流量特征（如早高峰、午高峰、晚高峰）；定期进行\n全链路压测，评估系统容量，确保在高负载下的稳定性； 根据\n容量模型，设置合理的限流策略。防止突发流量超出系统承受范\n围，保障服务稳定。\n 可观测性：明确业务的关键监控点和指标（成功量、成功率、响\n应时间等）。建立实时监控和预警机制，及时发现和处理异常情\n况。\n 变更防御：建立变更感知体系， 构建变更流程机制，对核心业\n务实施严格变更管理，确保安全可控，保证所有变更可被定位，\n有迹可循。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n353\nSRE 实践白皮书（2024 年）\n 应急能力：设定应急响应率目标，设定应急组织整形，提升团队\n快速定位和解决问题的应急止血能力，确保故障迅速恢复。\n 历史故障：对历史故障进行分析复盘， 找出稳定性薄弱环境，\n并进行跟进。\n在明确了关键业务场景后， 需要进行关键业务的 SLI / SLO\n体系的构建。\n5.1.5 应用服务 SLI/SLO/SLA\n5.1.5.1 什么是 SLI/SLO/SLA\n（1）SLI（Service Level Indicator，服务水平指标）是衡量\n系统或服务性能的具体指标，直接反映服务的质量或稳定性。SLI\n通常用来监控特定的服务性能数据，如可用性、延迟、错误率、吞\n吐量等。它是实现 SLO 的基础，通过 SLI 的实时数据监测，可以判\n断服务是否达到了预期的 SLO 目标。\n354\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n![Image 225](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_225.jpg)\nSRE 实践白皮书（2024 年）\n一般把这些指标分为 3 层：用户体验、中间件、基础设施。用\n户体验主要是指和用户强相关的一些功能监控，比如登录、购买等\n功能，这些往往用户的关键路径，也是我们应该关注的点；中间件\n主要指在架构中用到的开源组件等；基础设施就是指网络、物理服\n务器、物理电源、磁盘阵列卡等，在建设 SLO 时优先梳理用户体验\n层指标。\n（2）SLO（Service Level Objective，服务水平指标）是对服\n务质量的量化目标。它定义了系统或服务在一段时间内需要达到的\n性能或可靠性标准，通常用百分比来表示。例如，如果设定了 99%\n的 SLO，那么服务在一个月内至少要有 99%的时间可用，或其响应时\n间需要满足一定的标准。SLO 通常基于用户体验和业务需求来设\n定，既要满足用户期望，又要在技术和成本上实现可行。\n（3）SLA（Service Level Agreement，服务水平协议）是服务\n提供商与客户之间达成的正式协议，规定了服务的期望水平以及未\n达标时的补偿措施。SLA 通常会包括具体的 SLO 目标，例如服务可\n用性为 99.9%，以及服务中断的赔偿条款。SLA 的目标是明确责任和\n预期，确保服务提供方和客户之间的透明度和信任。\n在特定情况下，可以考虑将 SLA / SLO 与绩效挂钩，这样可\n以确保团队的目标与组织的服务质量目标一致。团队成员可能更加\n积极地工作以满足或超越这些目标。同时这样也为团队设定了明确\n的期望，并能实现数据驱动的评估方式。但是，与绩效挂钩，团队\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n355\nSRE 实践白皮书（2024 年）\n成员可能会过度专注于短期指标，而忽视长期目标和创新。建议每\n个组织根据客观的实际情况进行使用。\n5.1.5.2 如何建设 SLI/SLO\nSLO 建设步骤：确认 SLO 预期 -> 梳理关键链路 -> 梳理 SLI\n（1 原则+1 方法+关键链路） -> 上报 SLI -> 创建监控策略 -> 计\n算 SLO -> SLO 指标呈现\n1．确认预期\n和业务负责人、产品、策划、开发等相关干系人一起确认该功\n能的预期，即 SLO。需要注意的是，SLO 的设定需要根据服务实际需\n求来制定。趋近于 100%的 SLO，将会消耗巨大的成本，但是用户可\n能并感受不到变化。过低的 SLO，将会让用户的感受变差。我们需\n要确定在不影响用户体验的情况下，确定一个合理的 SLO。\n2．梳理关键路径\n梳理用户从功能入口到使用该功能的关键路径，比如：\napp 启动 --> app 更新检查 --> app 更新 --> 登录 --> 获\n取昵称 --> 展示昵称\n3．梳理 SLI\n原则：优先梳理用户体验层 SLI，即用户可以感受到的功能、\n按钮\n方法：VALET，从 V（容量）、A（可用率）、L（延迟）、E\n（错误）、T（人工提单，保底分类） 5 个维度来梳理，避免漏掉\nSLI\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n356\nSRE 实践白皮书（2024 年）\n将这 2 个原则套在第 2 步关键路径的每个节点，即可梳理出一\n份相对完整的 SLI 列表\n4．上报 SLI\n对于没有 SLI 需要客户端或服务端从新上报到监控系统，对于\n已有的 SLI 需要校验其准确性\n5．创建监控策略\n针对每个 SLI 需要创建一条监控策略，策略的告警阈值根据第\n一步定的 SLO 来确定。举个例子，若 SLO 定为 99%，则“app 更新检\n查成功率”指标的告警阈值为 < 99%\n6．计算 SLO\n围绕业务稳定性衡量，一共设计了 5 个指标, 多个维度，并基\n于监控系统计算出相关指标的值。以下的表格是一个指标和维度的\n事例的组合,供参考，通常 SLO 的指标维度，一定会包含时间维\n度，服务维度，其他可以根据自己的业务特性自行定义。\n维度信息\n维度\n含义\nVALET\nVALET 方 法 论 ， 分 别 代 容\n量、错误、延迟、可用率、工\n单\nrange_time\n固定的时间范围，单位：天\n枚举值\nVolume\nAvailability\nLatency\nError\nTicket\n1\n7\n30\n180\n365\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n357\nSRE 实践白皮书（2024 年）\nstrategy_name 监控策略名\nstrategy_id\n监控策略 ID\nbk_cc_biz_id\n业务 ID\nbk_biz_name\n业务名\n指标信息\n策略名\n策略 ID\n指标\n含义\n计算公式\n包 含 维\nSLO=（总时间-不\n稳定时间） / 总时\n间\n不稳定时间=多个\nSLI 告警时间并集\nslo_error_time_info\n=SUM （ SLI 告 警\n时间）\nslo\n汇 总 的 SLO 指\n标，代表当前场\n景的稳定性\nslo_error_t\nime_info\n各 个 SLI 错 误 消\n耗详情，根据该\n指标，可看出哪\n个指标的不稳定\n时间最长\n每 个 SLI 独 立 计\n算\nslo_error_t\nime\n汇总之后的错误\n消耗\n不稳定时间=多个\nSLI 告警时间并集\n度\nrange_time\nvelat\nrange_time\nstrategy_na\nme\nstrategy_id\nbk_cc_biz_i\nd\nbk_biz_nam\ne\nrange_time\nmttr\nmtbf\n告警平均恢复时\n间，值越小越好\nmttr = 故障总时间\n/ 故障次数\nrange_time\n平均告警间隔时\n间，值越大越好\nmtbf = 正常时间 /\n故障次数\nrange_time\n7．SLI/SLO 指标呈现\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n358\nSRE 实践白皮书（2024 年）\nSLO 的展示一定要基于对应 SLI，目的在于让相关干系人能直观\n的看到当前 SLI 下的 SLO、错误消耗等指标。展示的思路根据不同\n的用户，大致可以分为三种方式：\n（1）业务场景视角的展示。针对业务某一个 SLI 或者某个场景\n做深度的 SLO 数据展示。可以分为多个仪表盘，每个仪表盘再按照\nVELA 维度展示该功能所有 SLI 指标。达到效果：每个 SLI 的实时数\n据、可自定义时间范围展示 SLI 指标的曲线及具体值。更聚焦在某\n个场景或某个 SLI 的深度展示和分析。例如：某业务登录场景的\nVIP 延迟、鉴权成功率、用户信息拉取延迟等登录场景核心链路的\nSLO 详情展示。大多用于故障定位，排查问题。\n（2）业务整体视角的展示。从业务的角度，展示多个 SLI 的\nSLO 数据，将多个功能的 SLO 指标汇总在一起展示。达到效果：通\n过该仪表盘查看各个功能过去不同时间维度的 SLO，用于判断各个\n功能的稳定性（slo）；当某个 SLO 低于我们预期时，可以定位到\n具体影响的 SLI 指标（slo_error_time_info、slo_error_time）；\n各个功能的告警发生频率（mtbf）；各个功能的告警恢复时间\n（mttr）。例如：某业务的登录、支付、上传、浏览、语音等核心功\n能的 SLO 整体展示。大多用于单业务稳定性评估。\n（3）多业务整体视角的展示。从整个公司或集团的更高的视角\n展示多业务的 SLO 数据，将公司或集团主要业务模块 SLI 和 SLO 定\n义清晰，并展示在一个大屏上。例如：公司所有业务登录成功率、\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n359\nSRE 实践白皮书（2024 年）\n公司所有业务信息发送成功率、所有业务添加购物车成功率等。用\n于评估整体技术中台能力，整体服务质量。\n整个 SLI/SLO 的展示，从微观业务场景细节，到宏观业务基本\n状况，根据不同的使用场景，不同的用户需求，可以拆分为不同展\n示模式，汇总不同量级的数据，灵活展示。\n5.1.5.3 如何持续迭代 SLI/SLO\nSLO 出来后，可以用实际值和之前的预期做比对，分为以下 2\n种情况\n1．实际 SLO < 预期 SLO。比如实际值为 95%，预期为 99%\n此时可观察 SLI 的错误消耗，先对错误消耗最大的 SLI 进行调\n查，如果是指标质量问题导致 SLO 低于预期，那则修复 SLI 后再进\n行观察\n如果指标质量一些正常，也无错误数据，那说明指标的实际情\n况就是这样，确实达不到预期值。此时又可分为 2 种情况：\n业务架构不合理或部署方式不合理导致的 SLI 指标达不到预\n期：比如架构单点导致用户访问不稳定，集中部署导致用户访问延\n迟加大，此时应该利用 SLO 数据推动开发进行架构调整，推动运维\n进行布署优化，从而提升用户体验\nSLO 预期高于现实：可能现实条件确实达不到我们的要求，此\n时应该要放宽 SLO，降低 SLO 预期。否则我们的 SLO 指标会一直处\n于不达标状态，开发、运维也无能为力，这样的结果会造成团队恐\n慌，一定不是我们想要的。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n360\nSRE 实践白皮书（2024 年）\n2．实际 SLO > 预期 SLO。比如实际值为 99%，预期为 95%\nSLI 覆盖率低：确定关键链路 SLI 是否梳理完整，如果有缺\n失，则补齐，先提高 SLI 在关键路径的覆盖率\nSLO 预期太低：这么导致我们对用户体验的衡量不够准确，可\n能会出现用户体验很糟糕，但是我们 SLO 数据一切正常，导致与我\n们目的背道而驰。此时正确的措施应该是收紧 SLO，提高预期\n3．SLO 迭代小结\n从上面的分析来看，迭代 SLO 主要分为 2 大类：实际 SLO < 预\n期 SLO 和实际 SLO > 预期 SLO。\n实际 SLO < 预期 SLO：一般情况下我们首先要做是看为什么没\n有达到预期的 SLO，而不是调整 SLO 的数值。只有有充分理由说明\n之前的 SLO 制定存在不合理，那么我们才能放宽 SLO。\n实际 SLO > 预期 SLO：实际的 SLO 比预想的要好，我们可以分\n析是之前的目标设置的低了，还是我们投入了超过预期的成本。如\n果这个服务不需要这么高的 SLO，而我们也需要降低一些成本，那\n么可以适当放宽 SLO。\n对于 SLO 的持续迭代，可以从下 5 个步骤执行：\n SLI 质量优化\n SLI 覆盖率提升\n 架构调整\n 部署优化\n 放宽或收紧 SLO\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n361\nSRE 实践白皮书（2024 年）\n5.2 故障应急体系\n5.2.1 故障应急体系设计原则\n故障应急体系设计应以快速恢复业务为核心目标。故障应急生\n命周期通常包括故障发现、故障诊断、故障恢复、故障复盘四个阶\n段。通过完善的监控与发现机制、有效的故障诊断与定界、及时的\n恢复措施、全面的复盘与改进，再配套适当的组织流程、定期演练\n和持续优化，以及相关工程平台的落地，是实现故障应急响应能力\n提升、保障业务连续性的关键。\n\n基于上述背景，IT 运维部一直在思考如何建设一个变更一体化\n平台，抽象出四个“一体化”的思路：自研和外购一体化、传统和\n云原生一体化、测试和生产一体化、敏捷和安全一体化。\n（1）自研和外购一体化\n外购业务系统是由厂商维护的，厂商提供的就是一个制品包，\n因此需要支持厂商运维人员在平台维护程序包，并完成发布动作。\n变更一体化平台支持运维人员在界面上手动上传程序包，存储\n在共享存储服务中。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n327\n![Image 200](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_200.jpg)\n为了避免存储堆积，可以自定义设置存储清理策略：\n 永不清理\n 按天数清理\n 按版本清理\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n328\n![Image 201](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_201.jpg)\n![Image 201](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_201.jpg)\n在发布任务中，会将程序包存储的源服务器 IP 和源地址，以及\n分发的目标 IP 和目标路径，作为参数变量传给标准运维流程，这样\n可以通过作业平台的原子实现，文件的快速分发。\n标准运维中引用快速分发文件的原子，利用 GSE 的大文件分发能\n力，可以快速自动将文件分发到目标机器上。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n329\n![Image 203](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_203.jpg)\n自研业务系统是开发团队维护，通过流水线打包之后，自动入库\n到标准的制品仓库中，在测试环境和生产环境中都是存储在制品仓\n库中。为了维护制品包的一致性，变更一体化平台支持通过制品库\n插件，对接制品仓库同步程序包信息。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n330\n![Image 204](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_204.jpg)\n上传制品库插件\n针对这种类型，变更平台不会再存一份程序包，因此是调用制品\n库接口获取程序包的名称、版本、MD5、下载地址，用于做程序包分\n发。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n331\n![Image 205](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_205.jpg)\n![Image 205](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_205.jpg)\n![Image 205](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_205.jpg)\n考虑到直接开通所有主机到制品仓库的网络策略，存在较大的风\n险敞口，以及各个云区域到制品仓库的网络速度限制，不会在目标\n机器上直接执行 curl 命令下载程序包。\nSRE 同学编写了自定义的文件分发原子，用一台代理机器，和制\n品仓库在同一个云区域，先执行 curl 下载命令，将程序包从制品仓\n库下载到本地，再通过作业平台的文件分发功能，实现大文件的快\n速分发。\n通过兼容设计，将两种类型的制品包统一管理起来，用同一套发\n布逻辑，完成制品包的分发和部署，实现自研系统和外购系统的一\n体化。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n332\n![Image 208](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_208.jpg)\n（2）传统和云原生的一体化\n公司的传统应用部署在主机上，发布时需要登录主机操作，而\n云原生应用部署在 K8s 集群上，发布时需要执行 kubectl 命令完\n成。另外部分工具的主机发布任务和容器发布任务存在割裂，无法\n统一执行。而应用发布中心支持一个任务同时兼容主机发布和容器\n发布，实现一次审批一次编排。\n针对主机场景，使用管控平台纳管所有主机，标准运维编排所\n有主机发布任务。并且针对同类型的业务系统，规范建议复用相同\n的标准运维流程，无需每个业务团队自己建设。通用化规范化发布\n流程建议：\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n333\n![Image 209](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_209.jpg)\n![Image 209](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_209.jpg)\n(1)分发制品包：将具体的制品包名称、分发 IP、分发路径设\n置为变量，由业务系统在发布时传入，这样所有业务系统的制品分\n发复用这一个规范分发节点\n(2)备份程序：完成原有位置的程序备份\n(3)屏蔽告警策略：传入目标主机 IP，或者屏蔽策略 ID，停服\n务前自动完成告警消息屏蔽。\n由于变更时存在服务启停，变更期间必然产生大量无效告警，\n因此需要将变更目标主机的进场告警屏蔽。此操作也是引用一个通\n用的监控平台原子。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n334\n![Image 211](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_211.jpg)\n(4)停止服务：执行停服务脚本\n(5)替换程序包：替换程序包\n(6)重启服务：执行启服务脚本\n(7)端口检查：检查进程端口是否启动\n(8)地址检查：检查业务访问地址访问是否正常，编排一个通用\n检查脚本，具体 URL 通过参数变量传入，多业务系统复用此脚本\n(9)解除告警屏蔽：变更完成之后，自动解除告警屏蔽\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n335\n![Image 212](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_212.jpg)\n![Image 212](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_212.jpg)\n(10)发送通知\n针对云原生的应用，变更平台支持编排容器应用的 yaml 文件，\n提供两种编排方式，一种是原生管理方案，支持运维人员直接导入\nyaml 文件进行管理；一种是将 yaml 文件中的关键参数抽取成表单\n参数，运维人员只需要下拉选择即可，降低容器应用的编排门槛。\n做容器发布时需要调用 K8S 接口，变更平台通过 yaml 插件集成\n接口实现容器发布。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n336\n![Image 214](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_214.jpg)\n因此变更一体化平台，支持将传统主机发布节点，和云原生发布\n节点，编排到同一个发布任务中，实现“传统和云原生的一体\n化”。\n此外考虑到还存在数据库的 SQL 更新、纯配置更新等不同场景，\n变更平台在调度引擎时，单独设计的插件模块，支持通过可扩展的\n插件去接入更多发布类型。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n337\n![Image 215](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_215.jpg)\n![Image 215](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_215.jpg)\n（3）测试环境和生产环境的一体化\n为了提高生产环境的发布成功率，要求生产环境的发布操作，\n尽可能 1 比 1 复刻测试环境的发布操作，达到测试环境和生产环境\n的一体化。\nIT 运维部的 SRE 专家将发布操作拆解成三个环节，每个环节都\n需要保证测试环境和生产环境的一致性。\n环节一：应用拓扑\n在测试环境一般只会有一套集群，而生产环境两地三中心，至\n少存在三套集群，一般会认为测试和生产的环境没办法保持一致，\n从而忽略测试环境的维护。但实际上可以在应用拓扑层级保持一\n致，并且按照同一套规范去维护。\n公司在测试环境、预发环境、生产环境都使用“业务-子系统-\n集群-模块”四层拓扑架构，减少架构层面的疏漏。\n环节二：制品管理\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n338\n![Image 217](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_217.jpg)\n在测试环境和生产环境都部署独立制品仓库，开通从测试环境仓\n库到生产环境仓库的单向定点网络，并创建定期制品同步策略。\n开发人员在 CI 流水线阶段完成构建打包之后，就会自动入库，\n将版本包存储到制品仓库中；在测试人员完成测试后，自动触发制\n品晋级任务，自动将制品从测试环境的制品仓库同步到生产环境的\n制品仓库，用于生产变更。\n这样确保生产环境要发布的制品包，是经过测试，并且和测试\n环境的版本是一致的，减少运维人员在过程中的手动下载上传干\n预，全流程自动化流转。\n环节三：发布流程管理\n发布流程时生产环境变更中的最核心的部分，更要求生产环境\n的发布流程，是在测试环境中完整执行过的。且为了降低人工重新\n配置的复杂度以及出错概率，变更平台中的发布任务模板和标准运\n维执行流程，都是支持从测试环境导出，直接导入到生产环境，保\n证发布流程的一致性。\n变更一体化平台从应用拓扑、制品管理、发布流程管理三个维\n度，实现测试环境和生产环境的一致性，提高生产环境变更的稳定\n性。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n339\n（4）敏捷和安全的一体化\n在测试环境，开发人员通过 CI 流水线做持续集成和持续部署，\n实现快速开发迭代，因此对变更平台的要求是：能够作为 CD 节点完\n美到 CI 流水线中，实现应用的快速部署。\n变更平台开放了丰富的 OpenAPI 去满足 CI 流水线的要求：\nsearch_template 接口支持传入业务系统查询发布任务模板\ncreate_task 接口支持传入任务模板 id 创建发布任务\noperate_task 支持操作发布任务，包括执行、暂停、灰度、停\n止\n在生产环境，最重要的要求是稳定安全，因此需要对业务变更\n操作进行更多管控。一方面是在重要时间段内不允许变更，比如证\n券交易期间，重大会议期间；另外一方面所有变更操作需要和工单\n系统对接，运维领导审批通过之后才能执行发布。\n340\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n![Image 218](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_218.jpg)\n创建业务保障策略，在重大会议期间、节假日期间，从技术上\n限制运维人员对生产环境进行变更操作。目前大部分金融企业都是\n采用管理规范进行约束，在此期间不允许进行变更，但无法落地到\n技术层面进行强管控，还是存在业务团队冒险变更的风险。深层原\n因是，没有一体化的变更平台，每个业务团队使用各自的变更工具\n进行操作，无从限制。\n业务保障策略支持灵活配置满足不同场景需求：\n 支持设置策略状态，默认启用/停用\n 支持限制超管\n 支持选择生效业务，不同业务的安全级别不同，不同时间管\n控要求不一样，C 端业务强管控，内部系统弱管控；以业务\n系统级别管控，而不是具体 IP 维度管控，降低运维复杂度\n 支持自定义设置生效时间，日常证券开市时间是周期性周\n一-周五，节假日是固定时间段\n通过工单插件对接现有的变更工单系统，实现生产变更操作的\n审批。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n341\n![Image 219](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_219.jpg)\n在测试环境，通过 OpenAPI 集成到 DevOps 流水线中，实现快速\n发布，达到“敏捷”要求；在生产环境通过业务保障策略和工单审\n批策略，实现发布管控，达到“安全”要求。\n(三) 总结及展望\n变更一体化平台建设是一套长期动态的系统工程，随着应用系\n统及周边配套平台联动，还有很多工作在规划中，有很多能力在持\n续完善，例如度量、复盘、规范化等等。另外一些源自互联网的新\n方案例如 argoCD 等相对于传统的 K8s 部署方式集成度更高，可以\n探索其与合规跟质量的结合点。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n342\n5.1 故障体系\n5.1.1 故障定义\n故障的定义如下：在日常运营中，无论任何原因导致我们的服\n务中断、服务品质下降或用户服务体验下降，都被视为故障。但不\n包括用户环境活自身操作引起的问题。\n5.1.2 故障等级\n故障等级是衡量故障影响度的标准，数字越小级别越高。通常\n业界是以 P0 P1 P2 P3 P4 P5 等作为序列， P0 为影响度最高的故\n障（也有部分公司以 P1 作为最高级别）。\n影响度参考指标包括客户投诉量、资金损失金额、受影响用户\n数，故障时长，业务重要性等。\n通常情况下， 故障等级由 GOC、业务开发、质量团队、业务\nSRE 和业务稳定性等部门共同制定。而且相关故障等级需要提交业\n务负责人审批，确保业务方对相关故障级别标准的认可。并定期由\nGOC 统一审核故障定义和等级，并公示，确保大家按统一标准执\n行。\n故障等级的作用，我们可以基于这些定义，开展可观测性建设\n（配置针对性的告警体系以及预案）和平台能力构建（包括而不仅限\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n350\nSRE 实践白皮书（2024 年）\n于可用性架构设计及评估， 压测，混沌演练， 容灾演练），目的\n是提前识别潜在问题，进行预防和保障。\n在故障发生后，需要对故障进行评估，并采用就高原则， 如果\n一个案例符合多条故障定义，通常以最高等级为准。\n5.1.3 故障序列\n高稳定性团队可能会面临因非技术问题（如业务运营方配置失\n误，网络运营商的抖动或其他外部因素）引起的故障，因此需要明\n确技术故障与外部原因之间的区别，因此需要引入故障序列。\n故障序列是根据故障的根本原因和实际影响进行分类的方法，\n区分“故障根因”和“实际影响”。\n在定义故障序列时，主要考虑以下几点：\n技术责任：判断故障是否由技术原因引起，如是，则可能扣减\n故障预算。我们关注 P1 到 P5 等级的故障，因为它们通常涉及技\n术责任并影响业务。\n故障记录与配额：评估年度稳定性需考虑故障记录和配额。许\n多公司限制每年故障次数，超出后可能影响绩效。因此，非技术原\n因的故障，如业务操作失误，不扣减预算。例如，外部原因的故障\n标记为“W”，业务操作引起的标记为“S”。公司可根据这些分类\n制定定义更明确相关的稳定性指标。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n351\nSRE 实践白皮书（2024 年）\n5.1.4 关键业务场景\n组织有众多业务，究竟该优先保障哪些？我们应聚焦于最重要\n的业务，而非面面俱到。因此，我们定义了关键服务接口和结果\n点，即 关键业务场景， 也称 GOC（Global Operation Center）场\n景，也有公司叫 NOC (Network Operation Center） )。只要确保\n这些 GOC 场景的稳定，组织就不会出现重大问题。\n关键业务场景通过以下三个领域定义：\n 场景描述：明确需要保障的业务场景及其功能。\n 场景影响评价：设定衡量指标（如成功量、成功率、响应时间）\n来评估场景的稳定性和性能。可以理解为 SLI /SLO 体系的构\n建。\n 故障等级：明确场景故障时可能涉及的最高等级。重要业务需重\n点保障以避免高等级故障。\n实例说明\n以登录功能为例：\n免密登录：使用成功量作为衡量指标，若成功量下降，则可能\n触发相应故障等级。\n账号密码登录：使用成功率进行衡量。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n352\nSRE 实践白皮书（2024 年）\nGOC 场景定义的目的：\n通过定义 GOC 场景，聚焦关键服务接口和结果点，驱动和确保\n公司整体稳定性。\n围绕 GOC 场景，开展容量评估、压测、限流、监控、变更管理\n等各项工作。\n 业务链路： 清晰了解所保障业务的核心目标和完整的业务链\n路。明确稳定性保障团队，当问题发生时，能够迅速召集相关人\n员共同解决。\n 容量模型：从接入层到中间件，再到下游数据库，全面了解系统\n架构；建立性能基线， 明确资源成本；描绘流量模型，识别不\n同时间段的流量特征（如早高峰、午高峰、晚高峰）；定期进行\n全链路压测，评估系统容量，确保在高负载下的稳定性； 根据\n容量模型，设置合理的限流策略。防止突发流量超出系统承受范\n围，保障服务稳定。\n 可观测性：明确业务的关键监控点和指标（成功量、成功率、响\n应时间等）。建立实时监控和预警机制，及时发现和处理异常情\n况。\n 变更防御：建立变更感知体系， 构建变更流程机制，对核心业\n务实施严格变更管理，确保安全可控，保证所有变更可被定位，\n有迹可循。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n353\nSRE 实践白皮书（2024 年）\n 应急能力：设定应急响应率目标，设定应急组织整形，提升团队\n快速定位和解决问题的应急止血能力，确保故障迅速恢复。\n 历史故障：对历史故障进行分析复盘， 找出稳定性薄弱环境，\n并进行跟进。\n在明确了关键业务场景后， 需要进行关键业务的 SLI / SLO\n体系的构建。\n5.1.5 应用服务 SLI/SLO/SLA\n5.1.5.1 什么是 SLI/SLO/SLA\n（1）SLI（Service Level Indicator，服务水平指标）是衡量\n系统或服务性能的具体指标，直接反映服务的质量或稳定性。SLI\n通常用来监控特定的服务性能数据，如可用性、延迟、错误率、吞\n吐量等。它是实现 SLO 的基础，通过 SLI 的实时数据监测，可以判\n断服务是否达到了预期的 SLO 目标。\n354\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n![Image 225](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_225.jpg)\nSRE 实践白皮书（2024 年）\n一般把这些指标分为 3 层：用户体验、中间件、基础设施。用\n户体验主要是指和用户强相关的一些功能监控，比如登录、购买等\n功能，这些往往用户的关键路径，也是我们应该关注的点；中间件\n主要指在架构中用到的开源组件等；基础设施就是指网络、物理服\n务器、物理电源、磁盘阵列卡等，在建设 SLO 时优先梳理用户体验\n层指标。\n（2）SLO（Service Level Objective，服务水平指标）是对服\n务质量的量化目标。它定义了系统或服务在一段时间内需要达到的\n性能或可靠性标准，通常用百分比来表示。例如，如果设定了 99%\n的 SLO，那么服务在一个月内至少要有 99%的时间可用，或其响应时\n间需要满足一定的标准。SLO 通常基于用户体验和业务需求来设\n定，既要满足用户期望，又要在技术和成本上实现可行。\n（3）SLA（Service Level Agreement，服务水平协议）是服务\n提供商与客户之间达成的正式协议，规定了服务的期望水平以及未\n达标时的补偿措施。SLA 通常会包括具体的 SLO 目标，例如服务可\n用性为 99.9%，以及服务中断的赔偿条款。SLA 的目标是明确责任和\n预期，确保服务提供方和客户之间的透明度和信任。\n在特定情况下，可以考虑将 SLA / SLO 与绩效挂钩，这样可\n以确保团队的目标与组织的服务质量目标一致。团队成员可能更加\n积极地工作以满足或超越这些目标。同时这样也为团队设定了明确\n的期望，并能实现数据驱动的评估方式。但是，与绩效挂钩，团队\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n355\nSRE 实践白皮书（2024 年）\n成员可能会过度专注于短期指标，而忽视长期目标和创新。建议每\n个组织根据客观的实际情况进行使用。\n5.1.5.2 如何建设 SLI/SLO\nSLO 建设步骤：确认 SLO 预期 -> 梳理关键链路 -> 梳理 SLI\n（1 原则+1 方法+关键链路） -> 上报 SLI -> 创建监控策略 -> 计\n算 SLO -> SLO 指标呈现\n1．确认预期\n和业务负责人、产品、策划、开发等相关干系人一起确认该功\n能的预期，即 SLO。需要注意的是，SLO 的设定需要根据服务实际需\n求来制定。趋近于 100%的 SLO，将会消耗巨大的成本，但是用户可\n能并感受不到变化。过低的 SLO，将会让用户的感受变差。我们需\n要确定在不影响用户体验的情况下，确定一个合理的 SLO。\n2．梳理关键路径\n梳理用户从功能入口到使用该功能的关键路径，比如：\napp 启动 --> app 更新检查 --> app 更新 --> 登录 --> 获\n取昵称 --> 展示昵称\n3．梳理 SLI\n原则：优先梳理用户体验层 SLI，即用户可以感受到的功能、\n按钮\n方法：VALET，从 V（容量）、A（可用率）、L（延迟）、E\n（错误）、T（人工提单，保底分类） 5 个维度来梳理，避免漏掉\nSLI\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n356\nSRE 实践白皮书（2024 年）\n将这 2 个原则套在第 2 步关键路径的每个节点，即可梳理出一\n份相对完整的 SLI 列表\n4．上报 SLI\n对于没有 SLI 需要客户端或服务端从新上报到监控系统，对于\n已有的 SLI 需要校验其准确性\n5．创建监控策略\n针对每个 SLI 需要创建一条监控策略，策略的告警阈值根据第\n一步定的 SLO 来确定。举个例子，若 SLO 定为 99%，则“app 更新检\n查成功率”指标的告警阈值为 < 99%\n6．计算 SLO\n围绕业务稳定性衡量，一共设计了 5 个指标, 多个维度，并基\n于监控系统计算出相关指标的值。以下的表格是一个指标和维度的\n事例的组合,供参考，通常 SLO 的指标维度，一定会包含时间维\n度，服务维度，其他可以根据自己的业务特性自行定义。\n维度信息\n维度\n含义\nVALET\nVALET 方 法 论 ， 分 别 代 容\n量、错误、延迟、可用率、工\n单\nrange_time\n固定的时间范围，单位：天\n枚举值\nVolume\nAvailability\nLatency\nError\nTicket\n1\n7\n30\n180\n365\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n357\nSRE 实践白皮书（2024 年）\nstrategy_name 监控策略名\nstrategy_id\n监控策略 ID\nbk_cc_biz_id\n业务 ID\nbk_biz_name\n业务名\n指标信息\n策略名\n策略 ID\n指标\n含义\n计算公式\n包 含 维\nSLO=（总时间-不\n稳定时间） / 总时\n间\n不稳定时间=多个\nSLI 告警时间并集\nslo_error_time_info\n=SUM （ SLI 告 警\n时间）\nslo\n汇 总 的 SLO 指\n标，代表当前场\n景的稳定性\nslo_error_t\nime_info\n各 个 SLI 错 误 消\n耗详情，根据该\n指标，可看出哪\n个指标的不稳定\n时间最长\n每 个 SLI 独 立 计\n算\nslo_error_t\nime\n汇总之后的错误\n消耗\n不稳定时间=多个\nSLI 告警时间并集\n度\nrange_time\nvelat\nrange_time\nstrategy_na\nme\nstrategy_id\nbk_cc_biz_i\nd\nbk_biz_nam\ne\nrange_time\nmttr\nmtbf\n告警平均恢复时\n间，值越小越好\nmttr = 故障总时间\n/ 故障次数\nrange_time\n平均告警间隔时\n间，值越大越好\nmtbf = 正常时间 /\n故障次数\nrange_time\n7．SLI/SLO 指标呈现\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n358\nSRE 实践白皮书（2024 年）\nSLO 的展示一定要基于对应 SLI，目的在于让相关干系人能直观\n的看到当前 SLI 下的 SLO、错误消耗等指标。展示的思路根据不同\n的用户，大致可以分为三种方式：\n（1）业务场景视角的展示。针对业务某一个 SLI 或者某个场景\n做深度的 SLO 数据展示。可以分为多个仪表盘，每个仪表盘再按照\nVELA 维度展示该功能所有 SLI 指标。达到效果：每个 SLI 的实时数\n据、可自定义时间范围展示 SLI 指标的曲线及具体值。更聚焦在某\n个场景或某个 SLI 的深度展示和分析。例如：某业务登录场景的\nVIP 延迟、鉴权成功率、用户信息拉取延迟等登录场景核心链路的\nSLO 详情展示。大多用于故障定位，排查问题。\n（2）业务整体视角的展示。从业务的角度，展示多个 SLI 的\nSLO 数据，将多个功能的 SLO 指标汇总在一起展示。达到效果：通\n过该仪表盘查看各个功能过去不同时间维度的 SLO，用于判断各个\n功能的稳定性（slo）；当某个 SLO 低于我们预期时，可以定位到\n具体影响的 SLI 指标（slo_error_time_info、slo_error_time）；\n各个功能的告警发生频率（mtbf）；各个功能的告警恢复时间\n（mttr）。例如：某业务的登录、支付、上传、浏览、语音等核心功\n能的 SLO 整体展示。大多用于单业务稳定性评估。\n（3）多业务整体视角的展示。从整个公司或集团的更高的视角\n展示多业务的 SLO 数据，将公司或集团主要业务模块 SLI 和 SLO 定\n义清晰，并展示在一个大屏上。例如：公司所有业务登录成功率、\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n359\nSRE 实践白皮书（2024 年）\n公司所有业务信息发送成功率、所有业务添加购物车成功率等。用\n于评估整体技术中台能力，整体服务质量。\n整个 SLI/SLO 的展示，从微观业务场景细节，到宏观业务基本\n状况，根据不同的使用场景，不同的用户需求，可以拆分为不同展\n示模式，汇总不同量级的数据，灵活展示。\n5.1.5.3 如何持续迭代 SLI/SLO\nSLO 出来后，可以用实际值和之前的预期做比对，分为以下 2\n种情况\n1．实际 SLO < 预期 SLO。比如实际值为 95%，预期为 99%\n此时可观察 SLI 的错误消耗，先对错误消耗最大的 SLI 进行调\n查，如果是指标质量问题导致 SLO 低于预期，那则修复 SLI 后再进\n行观察\n如果指标质量一些正常，也无错误数据，那说明指标的实际情\n况就是这样，确实达不到预期值。此时又可分为 2 种情况：\n业务架构不合理或部署方式不合理导致的 SLI 指标达不到预\n期：比如架构单点导致用户访问不稳定，集中部署导致用户访问延\n迟加大，此时应该利用 SLO 数据推动开发进行架构调整，推动运维\n进行布署优化，从而提升用户体验\nSLO 预期高于现实：可能现实条件确实达不到我们的要求，此\n时应该要放宽 SLO，降低 SLO 预期。否则我们的 SLO 指标会一直处\n于不达标状态，开发、运维也无能为力，这样的结果会造成团队恐\n慌，一定不是我们想要的。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n360\nSRE 实践白皮书（2024 年）\n2．实际 SLO > 预期 SLO。比如实际值为 99%，预期为 95%\nSLI 覆盖率低：确定关键链路 SLI 是否梳理完整，如果有缺\n失，则补齐，先提高 SLI 在关键路径的覆盖率\nSLO 预期太低：这么导致我们对用户体验的衡量不够准确，可\n能会出现用户体验很糟糕，但是我们 SLO 数据一切正常，导致与我\n们目的背道而驰。此时正确的措施应该是收紧 SLO，提高预期\n3．SLO 迭代小结\n从上面的分析来看，迭代 SLO 主要分为 2 大类：实际 SLO < 预\n期 SLO 和实际 SLO > 预期 SLO。\n实际 SLO < 预期 SLO：一般情况下我们首先要做是看为什么没\n有达到预期的 SLO，而不是调整 SLO 的数值。只有有充分理由说明\n之前的 SLO 制定存在不合理，那么我们才能放宽 SLO。\n实际 SLO > 预期 SLO：实际的 SLO 比预想的要好，我们可以分\n析是之前的目标设置的低了，还是我们投入了超过预期的成本。如\n果这个服务不需要这么高的 SLO，而我们也需要降低一些成本，那\n么可以适当放宽 SLO。\n对于 SLO 的持续迭代，可以从下 5 个步骤执行：\n SLI 质量优化\n SLI 覆盖率提升\n 架构调整\n 部署优化\n 放宽或收紧 SLO\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n361\nSRE 实践白皮书（2024 年）\n5.2 故障应急体系\n5.2.1 故障应急体系设计原则\n故障应急体系设计应以快速恢复业务为核心目标。故障应急生\n命周期通常包括故障发现、故障诊断、故障恢复、故障复盘四个阶\n段。通过完善的监控与发现机制、有效的故障诊断与定界、及时的\n恢复措施、全面的复盘与改进，再配套适当的组织流程、定期演练\n和持续优化，以及相关工程平台的落地，是实现故障应急响应能力\n提升、保障业务连续性的关键。\n\n(4)变更操作安全(权限与审计)\n1)高危语句拦截。作业平台应提供可自定义的高危语句自动检\n测与拦截功能，以避免高危变更命令操作对生产环境造成故障。\n2）权限管理。作业平台作为企业统一作业变更操作底座平台，\n需要具备足够细颗粒度、多维度、灵活适配各类组织架构的权限设\n计，以保证安全的变更操作。从工程化落地的最佳实践讲，需要从\n整体 SRE 工程体系上进行权限中心的设计，作业平台的权限申请、\n审批、控制将由权限中心进行集中控制管理。\n3）作业审计。作业平台应提供所有通过页面执行、定时执行和\nAPI 调用等渠道触发的任务历史查看记录以便于进行审计或在故障\n诊断中分析 IT 变更对故障的影响。\n4.3.1 B 站变更防控的设计与实践\nSRE Elite 收录点评\n该平台利用了 trace 和 CMDB 资源拓扑信息，以关联和聚合应用\n服务的变更，能够追踪并识别变更对整个服务生态系统的潜在影\n响，有助于提高故障排查的效率和准确性。\n在企业内不同部门存在多套发布变更系统，且短期难以推倒统\n一重建的情况下，从变更防控的视角，补充建设防控平台，通过统\n一模型、熔断控制等方式，实现自动化的集中式变更管控。\nSRE Elite 收录点评\n1，这是一个由混合云 SRE 团队提供的案例，作为基础设施质量\n的把控者，对变更的计划性和标准作业流程要求严格，对于企业内\n私有云 SRE 管理团队有一定的参考价值。\n\nSRE 可靠性提升措施：\n1.使用统一账号系统，提供统一鉴权；\n2.元数据（含权限表、用户表、trigger 等）自动备份，一键\n恢复；\n3.集群化部署，主备容灾，使用云盘避免机器故障导致数据丢\n失；\n4.代码拉取、提交、删除、变更记录接入公司审计系统；\n5.人员离职、组织架构变更，邮件提醒权限删除；\n6.禁止供应商 office 网络连接 P4，只能在云桌面内部访问；\n7.p4auth 服务器单独架设，commit 异常，切换 replica；\n网址：SRE-Elite.com\n\n微服务架构与容器化部署：平台应支持基于微服务架构和容器\n化的部署方式，确保平台自身的高可用性和可扩展性。在云原生环\n境中，作业平台应能够自动伸缩，适应不同规模的负载需求。\n通过以上设计，SRE 团队能够打造一套高效、可靠的作业平\n台，有效支撑故障应急响应的各个环节，从而大幅提升系统的稳定\n性和业务的连续性。\n5.2.3.4 面向故障应急的风险管理平台\n技术风险管理平台应面向各个业务部门，涵盖了众多业务单元\n（BU）。他们均能在该平台上进行操作。产品形态主要服务于普通的\nSRE（Site Reliability Engineer）人员以及业务接口人员。稍后\n我会介绍我们的整体应急响应架构。针对不同的用户角色，我们设\n计了多种工作台形式。例如，一些负责 GOC 的同学或类似职能的同\n事可能会在系统中输入特定故障定义的场景，为此我们建立了风险\n运营机制。\n构建哪些能力应对故障响应\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n405\n![Image 230](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_230.jpg)\nSRE 实践白皮书（2024 年）\n1. 故障发现\n故障应急是对 SRE 体系综合能力的考验，涵盖信息采集、快速\n反应和决策、稳定准确的处置、资源整合能力及日常演练形成的流\n程体系。\n关键能力\n信息采集：及时、准确地获取系统状态和故障信息。\n快速反应和决策：迅速分析并做出决策，确保问题及时解决。\n稳定处置措施：执行稳定、准确的应急措施，确保系统尽快恢\n复。\n资源整合：整合内外部资源，迅速调动所需资源处理故障。\n日常演练：通过日常演练，形成规范的流程体系，提高团队应\n急效率。\n\n(4)变更操作安全(权限与审计)\n1)高危语句拦截。作业平台应提供可自定义的高危语句自动检\n测与拦截功能，以避免高危变更命令操作对生产环境造成故障。\n2）权限管理。作业平台作为企业统一作业变更操作底座平台，\n需要具备足够细颗粒度、多维度、灵活适配各类组织架构的权限设\n计，以保证安全的变更操作。从工程化落地的最佳实践讲，需要从\n整体 SRE 工程体系上进行权限中心的设计，作业平台的权限申请、\n审批、控制将由权限中心进行集中控制管理。\n3）作业审计。作业平台应提供所有通过页面执行、定时执行和\nAPI 调用等渠道触发的任务历史查看记录以便于进行审计或在故障\n诊断中分析 IT 变更对故障的影响。\n4.3.1 B 站变更防控的设计与实践\nSRE Elite 收录点评\n该平台利用了 trace 和 CMDB 资源拓扑信息，以关联和聚合应用\n服务的变更，能够追踪并识别变更对整个服务生态系统的潜在影\n响，有助于提高故障排查的效率和准确性。\n在企业内不同部门存在多套发布变更系统，且短期难以推倒统\n一重建的情况下，从变更防控的视角，补充建设防控平台，通过统\n一模型、熔断控制等方式，实现自动化的集中式变更管控。\nSRE Elite 收录点评\n1，这是一个由混合云 SRE 团队提供的案例，作为基础设施质量\n的把控者，对变更的计划性和标准作业流程要求严格，对于企业内\n私有云 SRE 管理团队有一定的参考价值。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n265\n![Image 132](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_132.jpg)\n2，大量使用了 IaC 的方式对基础架构进行管理及变更，其中\n中移互联网有限公司（以下简称“中移互联”）是中国移动集\n团集团公司面向互联网领域设立的专业子公司，目前运营着多款拥\n有亿级用户规模的产品，包括移动云盘、移动认证、超级 SIM 等。\n从业务上看，这些业务有持续迭代发版的内在需求，然而，各业务\n线的发布变更能力存在显著差异，手工与工具并存，工具平台多样\n化，不仅效率低下，且容易出错，发布能力与业务发展水平不匹\n配。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n306\n为了解决上述问题，中移互联提出了同推动构建公司级统一应\n用敏捷发布平台。该平台旨在实现传统主机和云原生环境的全场景\n自动化发布变更，构建覆盖测试、灰度和生产环境的一体化应用发\n布能力，从而推动发布工作从手工到自动化、发布工具从分散化到\n集约化的转型，提升生产环境的变更成功率和发布效率，为业务团\n队提供稳定、安全的发布保障，进一步推动公司的数字化转型和业\n务发展。\n\n微服务架构与容器化部署：平台应支持基于微服务架构和容器\n化的部署方式，确保平台自身的高可用性和可扩展性。在云原生环\n境中，作业平台应能够自动伸缩，适应不同规模的负载需求。\n通过以上设计，SRE 团队能够打造一套高效、可靠的作业平\n台，有效支撑故障应急响应的各个环节，从而大幅提升系统的稳定\n性和业务的连续性。\n5.2.3.4 面向故障应急的风险管理平台\n技术风险管理平台应面向各个业务部门，涵盖了众多业务单元\n（BU）。他们均能在该平台上进行操作。产品形态主要服务于普通的\nSRE（Site Reliability Engineer）人员以及业务接口人员。稍后\n我会介绍我们的整体应急响应架构。针对不同的用户角色，我们设\n计了多种工作台形式。例如，一些负责 GOC 的同学或类似职能的同\n事可能会在系统中输入特定故障定义的场景，为此我们建立了风险\n运营机制。\n小米的最新战略目标是构建一个“人-车-家”全生态系统\n（Human x Car x Home），旨在满足全行业的需求。我们希望在这个\n生态系统中，能够像星辰大海一样广泛覆盖，做好每一个细节。为\n了体现我们小米 SRE 工作的复杂度，以下是一些关键数据：\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n402\nSRE 实践白皮书（2024 年）\n 我们拥有 6.99 亿台智能设备。\n 涵盖 200 多个智能硬件品类（仅小米品牌，不包括小米生\n态链）。\n 小米生态链大约覆盖了用户生活场景的 95%。\n小米 SRE 面临的挑战\n小米作为全球第三的手机制造厂商&全球领先的消费级 AIoT 平\n台，SRE 挑战巨大\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n403\n![Image 227](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_227.jpg)\n![Image 227](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_227.jpg)\nSRE 实践白皮书（2024 年）\n 目前有 7 亿联网设备（国内和国际）。\n 米家平台的月活跃用户数（MAU）为 760 万，作为互联互\n通的中枢。\n 小爱的月活跃用户数为 1.14 亿。小爱不仅仅是一个音\n箱，其后端也在适应 ChatGPT 的大潮流。\n 我们有 20 多个业务线，可以认为是集团级的组织单元。\n 全球范围内有 12 个核心机房，服务于各个地区。\n 使用了七家公有云，阿里云，金山云，腾讯云，火山云，\nAWS，Azure，GCP，积累了丰富的多云管理经验， 但是在过程\n中也因为各云标准不统一在对接过程中非常痛苦\n在故障应急方面，我们的目标是做到快速发现、快速定位和快\n速控制损失。具体而言，阿里的专家提出的经典目标是：1 分钟内\n发现故障，5 分钟内定位故障，10 分钟内控制损失。这也是我们\nSRE 团队的追求目标。\n数字化监控平台的核心能力三个解决方案\n稳定性保障解决方案\n解决方案，将安全生产稳定性保障左移，在入网控制时介入，\n对入网控制、发布上线、故障预防、故障发现、故障定位，故障恢\n复、故障改进提供端到端工具支撑。\n网址：SRE-Elite.com\n微信：SRE 精英联盟\n436\n![Image 259](/Users/allwefantasy/projects/mcp-docs/_images/SRE Whitepaper 1.0.5.pdf/image_259.jpg)\nSRE 实践白皮书（2024 年）\n生产运营保障体系结构框架\n生产安全保障体系：一个目标，依托四大保障，聚焦研运流程\n中十二项核心工作，严格把控七个关口。\n设计关关键点：架构设计阶段，确保包含冗余设计、灾备设\n计、熔断限流等稳定性设计。\n验证关关键点：上线前进行严格的版本管理和验证，确保系统\n符合稳定性和安全性要求。\n上线关关键点：确保研发与运维之间的无缝交接，规范上线流\n程，确保系统平稳上线。\n437\n\n"

p = """
使用以上文档来回答用户的问题。回答要求：

1. 严格基于文档内容回答        
- 如果文档提供的信息无法回答问题,请明确回复:"抱歉,文档中没有足够的信息来回答这个问题。" 
- 不要添加、推测或扩展文档未提及的信息

2. 格式如 ![image](./path.png) 的 Markdown 图片处理
- 根据Markdown 图片前后文本内容推测改图片与问题的相关性，有相关性则在回答中输出该Markdown图片路径。尽量保留图片输出。
- 根据相关图片在文档中的位置，自然融入答复内容,保持上下文连贯
- 完整保留原始图片路径,不省略任何部分

3. 回答格式要求
- 使用markdown格式提升可读性

问题：找到 SRE  可靠性架构设计 相关的内容
"""

r = llm.chat_oai(conversations=[
  {
    "role": "user",
    "content": docs + p
  }
])

print(r[0].output)


2024-12-18 10:04:01.061 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-12-18 10:04:01,116	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-12-18 10:04:01,144	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


In [ ]:

rag = LongContextRAG(llm,args=AutoCoderArgs(),tokenizer_path=VariableHolder.TOKENIZER_PATH)


rag._answer_question(question)

In [ ]:
# Clean up
import shutil
shutil.rmtree(test_dir)
print(f"Cleaned up test directory: {test_dir}")